# Utils Notebook
El propósito de este notebook es guardar todas las funciones auxiliares para la implementación del código principal. Se podrá encontrar organizado en diferentes secciones.

## Set-up

### Installs

In [ ]:
!pip install contractions
!pip install nltk
!pip install textblob
!pip install ffmpeg-python

### Imports

In [ ]:
import pandas as pd
import numpy as np
import datetime
import re
import os
import json
import contractions
from itertools import product
from textblob import TextBlob
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import string
from transformers import BertTokenizer, RobertaTokenizer, XLNetTokenizer, AutoModel, AutoTokenizer, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, logging
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision.models import VGG16_Weights
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import label_binarize
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

import ffmpeg
import librosa
import io
import warnings
import librosa.display
from scipy.ndimage import zoom

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


logging.set_verbosity_error()

### Diccionarios y variables globales

In [ ]:
# Diccionario global con paths base por dataset
DATASET_PATHS = {
    'MELD': '/kaggle/input/meld-dataset/MELD-RAW/MELD.Raw/',
    'MOSEI': '/kaggle/input/cmu-mosei/CMU-MOSEI-20230514T151450Z-001/CMU-MOSEI',
    'MOSEI_audios': '/kaggle/input/d/maunberg/cmu-mosei/Audio/Audio/WAV_16000/'
}

# Stopwords de NLTK
stop_words = set(stopwords.words("english"))

# Tokenizadores para el estudio del sentimiento
tokenizers = {
    "bert": BertTokenizer.from_pretrained("bert-base-uncased"),
    "bertweet": AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis"),
    "roberta": RobertaTokenizer.from_pretrained("roberta-base"),
    "twitter-roberta": AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment"),
    "xlnet": XLNetTokenizer.from_pretrained("xlnet-base-cased"),
}

## Lectura de CSVs

In [ ]:
def read_csvs(file_paths, dataset_type):
    """
    Función que lee múltiples archivos CSV desde una ruta base asociada al tipo de dataset.

    Args:
        file_paths (list): Lista de rutas a los archivos CSV que se deben leer.
        dataset_type (str): Indetificador del tipo de dataset, utilizada para obtener la ruta base.

    Returns:
        tuple of pandas.DataFrame: Tupla que contiene un Pandas DataFrame por cada archivo leído.

    Raises:
        ValueError: Si el tipo de dataset no está definido en DATASET_PATHS.
    """
    # Path inicial del dataset escogido
    input_path = DATASET_PATHS.get(dataset_type)

    if not input_path:
        raise ValueError(f"Dataset no reconocido: {dataset_type}")

    # Lectura de todos los CSVs de la lista
    dfs = [pd.read_csv(os.path.join(input_path, f)) for f in file_paths]

    # Retorno de los dataframes
    return tuple(dfs)

In [ ]:
def shape_dfs(dfs, name_dfs):
    """
    Función que muestra por consola las dimensiones (filas y columnas) de cada DataFrame proporcionado.

    Args:
        dfs (list of pandas.DataFrame): Lista de DataFrames cuyas dimensiones se desean mostrar.
        name_dfs (list of str): Lista de nombres que identifican a cada DataFrame.

    Returns:
        None: Esta función no retorna ningún valor; imprime resultados por consola.
    """
    # Por cada dataframe, mostramos sus dimensiones 
    for i in range(len(name_dfs)):
        print(f"{name_dfs[i]} tiene {dfs[i].shape[0]} filas y {dfs[i].shape[1]} columnas")


## Limpieza de CSVs

In [ ]:
def convert_time_to_seconds(time_str):
    """
    Función que convierte una cadena de tiempo en formato 'HH:MM:SS,MS' a segundos como número decimal.

    Args:
        time_str (str): Cadena de texto que representa el tiempo de comienzo o fin del clip de audio/video.

    Returns:
        float: Tiempo expresado en segundos.
    """
    # Se reemplazan las comas por puntos decimales
    time_str = time_str.replace(',', '.')
    
    # Se transforma el formato HH:MM:SS.MS a segundos
    h, m, s = time_str.split(':')
    s, ms = s.split('.')
    total_seconds = int(h) * 3600 + int(m) * 60 + int(s) + int(ms)/1000
    return total_seconds

def convert_time_columns_MELD(df):
    """
    Función que convierte las columnas 'StartTime' y 'EndTime' de un DataFrame MELD a segundos en formato numérico.

    Args:
        df (pandas.DataFrame): DataFrame del dataset MELD que contiene las columnas 'StartTime' y 'EndTime'.

    Returns:
        pandas.DataFrame: DataFrame con las nuevas columnas 'StartTime_sec' y 'EndTime_sec', y sin las columnas originales de tiempo.
    """
    # Se convierten las columnas de tiempo a valores numéricos
    df.loc[:, 'StartTime_sec'] = df['StartTime'].apply(convert_time_to_seconds)
    df.loc[:, 'EndTime_sec'] = df['EndTime'].apply(convert_time_to_seconds)
    
    # Se eliminan las columnas innecesarias
    df = df.drop(columns = ['StartTime', 'EndTime'])
    return df

def encode_MOSEI_video_column(df_list):
    """
    Función que codifica los nombres de los videos en los DataFrames del dataset MOSEI a valores numéricos.

    Args:
        df_list (list of pandas.DataFrame): Lista con tres DataFrames MOSEI que contienen la columna 'video'.

    Returns:
        tuple of pandas.DataFrame: Los tres DataFrames MOSEI originales, cada uno con una nueva columna 'video_encoded'.

    Notes:
        Se utiliza LabelEncoder de scikit-learn para asignar valores únicos a cada nombre de video. El encoder se entrena con la unión de los valores únicos de todos los DataFrames.
    """
    # Se unen los nombres de los videos de los 3 datasets MOSEI
    all_videos = pd.concat([df['video'] for df in df_list]).astype(str).unique()

    # Se define y entrena el encoder
    encoder = LabelEncoder()
    encoder.fit(all_videos)

    # Cada nombre de video en cada df se codifica a un valor numérico
    for df in df_list:
        df.loc[:, 'video_encoded'] = encoder.transform(df['video'].astype(str))

    return df_list[0], df_list[1], df_list[2]

In [ ]:
def clean_dfs(df, dataset_type, split_type):
    """
    Función que limpia un DataFrame eliminando valores nulos y redirige a la limpieza específica según el tipo de dataset.

    Args:
        df (pandas.DataFrame): DataFrame que se desea limpiar.
        dataset_type (str): Tipo de dataset, debe ser 'MELD' o 'MOSEI'.
        split_type (str): Tipo de partición del dataset ('train', 'dev', 'test').

    Returns:
        pandas.DataFrame: DataFrame limpio, procesado según las reglas específicas del dataset indicado.

    Raises:
        ValueError: Si el tipo de dataset no es 'MELD' ni 'MOSEI'.
    """
    # Se eliminan las filas con cualquier valor nulo
    df.replace('', pd.NA, inplace = True)
    df.dropna(inplace = True)

    # Se redirecciona a la limpieza pertinente
    if dataset_type == 'MELD':
        input_path = DATASET_PATHS.get(dataset_type)
        return clean_MELD_dfs(df, split_type, input_path)

    elif dataset_type == 'MOSEI':
        input_path = DATASET_PATHS.get(dataset_type + '_audios')
        return clean_MOSEI_dfs(df, input_path)

    else:
        raise ValueError("dataset_type debe ser 'MELD' o 'MOSEI'")

def clean_MELD_dfs(df, split_type, input_path):
    """
    Función que limpia y transforma un DataFrame del dataset MELD, eliminando columnas innecesarias, renombrando variables,
    generando rutas de video y convirtiendo columnas de tiempo a segundos.

    Args:
        df (pandas.DataFrame): DataFrame del dataset MELD que se desea procesar.
        split_type (str): Tipo de partición ('train' o 'dev'), utilizado para construir la ruta a los archivos de video.
        input_path (str): Ruta base del dataset MELD.

    Returns:
        pandas.DataFrame: DataFrame transformado con las columnas relevantes, rutas de video y tiempos en formato numérico.

    Raises:
        ValueError: Si `split_type` no es 'train' ni 'dev'.
    """
    # Se eliminan las columnas innecesarias
    df = df.drop(columns = ['Sr No.', 'Speaker', 'Episode'])

    # Se renombra la columna 'Utterance' como 'text'
    df = df.rename(columns={'Utterance': 'text', 'Sentiment': 'sentiment'})
    
    # Extensión del path según el dataframe
    if split_type == 'train':
        video_folder = 'train/train_splits/dia'
    elif split_type == 'dev':
        video_folder = 'dev/dev_splits_complete/dia'
    else:
        raise ValueError("split_type debe ser 'train' o 'dev' para MELD")

    # Se construye la columna 'video_path'
    df['video_path'] = (
        input_path + 
        video_folder + 
        df['Dialogue_ID'].astype(str) + '_utt' + 
        df['Utterance_ID'].astype(str) + '.mp4')

    # Se convierten las columnas de tiempo en formato numérico (segundos)
    df = convert_time_columns_MELD(df)
    return df

def clean_MOSEI_dfs(df, input_path):
    """
    Función que limpia y transforma un DataFrame del dataset MOSEI, eliminando columnas no relevantes, generando variables auxiliares
    y construyendo las rutas de acceso a archivos de audio.

    Args:
        df (pandas.DataFrame): DataFrame del dataset MOSEI que se desea procesar.
        input_path (str): Ruta base donde se encuentran los archivos de audio.

    Returns:
        pandas.DataFrame: DataFrame transformado con las columnas necesarias y nuevas variables generadas.
    """
    # Se eliminan las columnas innecesarias
    df = df.drop(columns = ['ASR'])
    
    # Se crea la columna 'neutral' basada en otras emociones
    emotion_cols = ['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear']
    df['neutral'] = df[emotion_cols].sum(axis = 1).apply(lambda x: 3 if x == 0 else 0)

    # Construcción de las columnas 'video_path' y 'audio_name'
    df['video_path'] = (
        input_path + df['video'].astype(str) + '.wav'
    )
    df['audio_name'] = (
        df['video'].astype(str) + '_' + df['start_time'].astype(str) + '_' + df['end_time'].astype(str) + '.wav'
    )
    return df


In [ ]:
def check_empty_text_rows(df):
    """
    Función que identifica y muestra la cantidad de filas con textos vacíos o nulos en un DataFrame, y devuelve dichas filas.

    Args:
        df (pandas.DataFrame): DataFrame a evaluar.

    Returns:
        pandas.DataFrame: Subconjunto del DataFrame original que contiene las filas con textos vacíos o valores nulos.

    Notes:
        Se imprime por consola el número de textos vacíos, nulos y el total de casos problemáticos detectados.
    """
    # Se detectan y cuentan las filas con textos vacíos
    empty_mask = df['text'].astype(str).str.strip() == ''
    num_empty = empty_mask.sum()

    # Se cuentan las filas con textos nulos
    num_null = df['text'].isnull().sum()

    # Se muestra un mensaje con el número de filas con textos vacíos por tipo y el total
    print(f"Textos vacíos (solo espacios o ''): {num_empty}")
    print(f"Textos nulos (NaN): {num_null}")
    print(f"Total de textos problemáticos: {num_empty + num_null}")

    # Se devuleven las filas con textos vacíos
    return df[empty_mask | df['text'].isnull()]

In [ ]:
def remove_empty_text_rows(df, name_df):
    """
    Función que elimina las filas con textos vacíos en la columna 'text' de un DataFrame y muestra un resumen de la operación.

    Args:
        df (pandas.DataFrame): DataFrame del que se desean eliminar las filas con texto vacío.
        name_df (str): Nombre del DataFrame.

    Returns:
        pandas.DataFrame: DataFrame resultante tras eliminar las filas con textos vacíos.
    """
    # Se calcula el número de filas original del dataframe
    original_len = len(df)

    # Se eliminan las filas cuyo texto esté vacío
    df = df[df['text'].astype(str).str.strip() != '']

    # Se calcula el número de filas final del dataframe y se muestra el mensaje
    final_len = len(df)
    print(f"Se han eliminado {original_len - final_len} filas con texto vacío. {name_df} ahora tiene un shape {df.shape}")
    return df

## Adecuación de las columnas de los CSVs

In [ ]:
def normalize_emotion_and_sentiment(df, dataset_type):
    """
    Función que normaliza las etiquetas de sentimiento y emociones en un DataFrame, adaptándolas según el tipo de dataset.

    Args:
        df (pandas.DataFrame): DataFrame que contiene las columnas de sentimiento y emociones a normalizar.
        dataset_type (str): Tipo de dataset al que pertenece el DataFrame ('MELD' o 'MOSEI').

    Returns:
        tuple:
            pandas.DataFrame: DataFrame con las etiquetas de sentimiento y emociones normalizadas.
            dict or None: Diccionario de mapeo de emociones usado en MELD, o None si no aplica.

    Notes:
        Se invoca `classify_sentiment` para estandarizar la columna de sentimiento y `normalize_emotions` para adaptar
        las emociones según el dataset. Si se trata de MELD, se imprime el diccionario de emociones transformadas.
    """
    # Normalización de la columna Sentiment
    df = classify_sentiment(df, dataset_type)
    
    # Normalización y adecuación de las columnas de emociones
    df, emotion_dict_MELD = normalize_emotions(df, dataset_type)

    # Print de la adecuación de las emociones de MELD
    if emotion_dict_MELD:
        print("Emociones en MELD:", emotion_dict_MELD)

    return df, emotion_dict_MELD if emotion_dict_MELD else None


def classify_sentiment(df, dataset_type):
    """
    Función que clasifica y normaliza los valores de la columna 'sentiment' en un DataFrame, asignando clases numéricas según el tipo de dataset.

    Args:
        df (pandas.DataFrame): DataFrame que contiene la columna 'sentiment' con valores categóricos (MELD) o numéricos (MOSEI).
        dataset_type (str): Tipo de dataset ('MELD' o 'MOSEI') que determina el esquema de codificación a utilizar.

    Returns:
        pandas.DataFrame: DataFrame con la columna 'sentiment' convertida a clases numéricas: 0 (negativo), 1 (neutral), 2 (positivo).

    Notes:
        - Para MELD, se mapean las categorías de texto ('negative', 'neutral', 'positive') a enteros.
        - Para MOSEI, los valores numéricos se agrupan en tres clases en función de su signo.
    """
    if dataset_type == 'MELD':
        # Se mapean los sentimentos de MELD a 3 clases
        sentiment_map_MELD = {'negative': 0, 'neutral': 1, 'positive': 2}
        # Se sustituyen los valores categóricos de la columna sentiment por valores numéricos
        df['sentiment'] = df['sentiment'].map(sentiment_map_MELD)

    elif dataset_type == 'MOSEI':
        # Se define una sub-función para clasificar el sentimiento de MOSEI
        def sentiment_map_MOSEI(value):
            # Si el valor es negativo, será la clase 0
            if value < 0:
                return 0
            # Si el valor es neutral, será la clase 1
            elif value == 0:
                return 1
            # Si el valor es positivo, será la clase 2
            else:
                return 2
        # Se aplica el cambio del sentimiento a clases
        df.loc[:, 'sentiment'] = df['sentiment'].apply(sentiment_map_MOSEI)

    return df
    

def normalize_emotions(df, dataset_type):
    """
    Función que normaliza las emociones en un DataFrame según el tipo de dataset, aplicando codificación numérica o escalado.

    Args:
        df (pandas.DataFrame): DataFrame que contiene las emociones a procesar.
        dataset_type (str): Tipo de dataset ('MELD' o 'MOSEI') que determina el tipo de normalización aplicada.

    Returns:
        tuple:
            pandas.DataFrame: DataFrame con las emociones normalizadas.
            dict or None: Diccionario de codificación de emociones utilizado (solo para MELD), o None si no aplica.

    Notes:
        - Para MELD, se reemplazan los valores categóricos de la columna 'Emotion' por valores numéricos entre 0 y 6.
        - Para MOSEI, cada columna de emoción ('happy', 'sad', etc.) se escala entre 0.0 y 1.0 usando normalización min-max, redondeada a un decimal.
    """
    # Definición del diccionario de emociones para MELD
    emotion_dict_MELD = None

    if dataset_type == 'MELD':
        # Lista de emociones de MELD
        emotions = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
        emotion_dict_MELD = {emotion: i for i, emotion in enumerate(emotions)}

        # Se reemplazan los valores categóricos de la columna Emotion por valores numéricos
        df['Emotion'] = df['Emotion'].map(emotion_dict_MELD)

    elif dataset_type == 'MOSEI':
        # Lista de emociones de MOSEI
        emotions = ['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear', 'neutral']
        # Por cada emoción, se normaliza su columna
        for emotion in emotions:
            min_val = df[emotion].min()
            max_val = df[emotion].max()
            range_val = max_val - min_val
            df[emotion] = round((df[emotion] - min_val) / range_val, 1) if range_val != 0 else 0.0

    return df, emotion_dict_MELD


## Preprocesamiento del dataset

### Preprocesamiento del texto

#### Preprocesamiento básico

In [ ]:
def preprocess_text_column(df):
    """
    Función que aplica las transformaciones de preprocesamiento sobre la columna 'text' de un DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame a preprocesar.

    Returns:
        pandas.DataFrame: DataFrame con la columna 'text' transformada tras aplicar limpieza y normalización textual.

    Notes:
        Las transformaciones aplicadas incluyen:
        - Conversión del texto a minúsculas.
        - Eliminación de repeticiones de letras y onomatopeyas.
        - Eliminación de símbolos extraños.
        - Expansión de contracciones.
        - Eliminación de signos de puntuación.
    """
    # Se pone el texto en minúsculas
    df['text'] = df['text'].apply(lambda x: x.lower())

    # Se limpian las palabras repetidas y las onomotopeyas
    df['text'] = df['text'].apply(clean_repetitions_and_sounds)      

    # Se eliminan los símbolos extraños
    df['text'] = df['text'].apply(remove_symbols)

    # Se expanden las contracciones
    df['text'] = df['text'].apply(expand_contractions)

    # Se eliminan los símbolos de puntuación
    df['text'] = df['text'].apply(remove_punctuation)

    return df

def clean_repetitions_and_sounds(text):
    """
    Función que limpia repeticiones innecesarias de palabras y elimina onomatopeyas o sonidos entre paréntesis en un texto.

    Args:
        text (str): Cadena de texto que se desea limpiar.

    Returns:
        str: Texto procesado, sin repeticiones consecutivas ni sonidos escritos entre paréntesis.
    """
    # Se cambian repeticiones por una única palabra (ex: "hey-hey-hey" -> "hey")
    text = re.sub(r'\b(\w+)(-\1)+\b', r'\1', text)
    
    # Se cambian repeticiones por una única palabra (ex: "hey hey hey" -> "hey")
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

    # Se eliminan las onomatopeyas y sonidos entre paréntesis (ex: (uhh), (stutter))
    text = re.sub(r'\(\s*[^)]+\s*\)', '', text)

    # Se eliminan los espacios duplicados creados después de limpiar
    text = re.sub(r'\s{2,}', ' ', text).strip()
    return text


def remove_symbols(text):
    """
    Función que elimina caracteres no ASCII de una cadena de texto.

    Args:
        text (str): Cadena de texto a procesar.

    Returns:
        str: Texto resultante sin símbolos ni caracteres especiales fuera del rango ASCII estándar.
    """
    # Se eliminan los caracteres no ASCII
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text


def expand_contractions(text):
    """
    Función que expande las contracciones en inglés presentes en una cadena de texto.

    Args:
        text (str): Cadena de texto que puede contener contracciones del inglés.

    Returns:
        str: Texto con las contracciones expandidas a su forma completa.

    Notes:
        Utiliza la librería `contractions` para convertir las expresiones.
    """
    # Se expanden las contracciones inglesas (ex: they're -> they are)
    return contractions.fix(text)


def remove_punctuation(text):
    """
    Función que elimina los signos de puntuación de una cadena de texto y corrige los espacios sobrantes.

    Args:
        text (str): Cadena de texto que se desea limpiar.

    Returns:
        str: Texto sin signos de puntuación ni espacios duplicados.

    Notes:
        Se eliminan todos los caracteres definidos en `string.punctuation`.
    """
    # Se eliminan los símbolos de puntuación
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Se eliminan los espacios duplicados creados después de limpiar
    text = re.sub(r'\s+', ' ', text).strip()
    return text


#### Stopwords en formato n-gramas y unitarias

In [ ]:
''' N-GRAMAS DE TIPO STOPWORD'''

def load_texts_from_dfs(dfs):
    """
    Función que extrae y concatena todos los textos no nulos de la columna 'text' a partir de una lista de DataFrames.

    Args:
        dfs (list of pandas.DataFrame): Lista de DataFrames.

    Returns:
        list of str: Lista de cadenas de texto resultantes, convertidas a tipo `str` y excluyendo los valores nulos.
    """
    # Se concatenan todos los textos disponibles
    return [str(row) for df in dfs for row in df['text'].dropna()]

def clean_and_tokenize(text):
    """
    Función que tokeniza un texto eliminando los símbolos y conservando únicamente las palabras que estén en una lista de 'stop_words'.

    Args:
        text (str): Cadena de texto que se desea tokenizar y filtrar.

    Returns:
        str: Cadena de texto reconstruida a partir de las palabras reconocidas como 'stop_words'.
    """
    # Sokenizan las palabras sin simbolos
    tokens = re.findall(r'\b\w+\b', text)
    return ' '.join([token for token in tokens if token in stop_words])

def extract_stopword_ngrams_corpus(corpus, min_df = 5, ngram_range = (2, 6)):
    """
    Función que extrae n-gramas compuestos únicamente por 'stopwords' desde un corpus textual.

    Args:
        corpus (list of str): Lista de cadenas de texto que componen el corpus.
        min_df (int, optional): Frecuencia mínima con la que un n-grama debe aparecer para ser considerado. Por defecto es 5.
        ngram_range (tuple of int, optional): Rango de tamaño de los n-gramas a extraer (mínimo, máximo). Por defecto es (2, 6).

    Returns:
        list of str: Lista de n-gramas ordenados de mayor a menor según la cantidad de palabras que los componen.

    Notes:
        - Se utiliza la función `clean_and_tokenize` para generar un corpus filtrado que conserve solo 'stopwords'.
        - Los n-gramas se extraen mediante `CountVectorizer` de scikit-learn.
        - Los n-gramas que no cumplen con el umbral de frecuencia `min_df` son descartados.
    """
    # Se crea el corpus limpio sin símbolos
    cleaned_corpus = [clean_and_tokenize(text) for text in corpus]

    # Se vectorizan los n-gramas para quedarnos aquellos se repitan un mínimos de veces en el df
    vectorizer = CountVectorizer(ngram_range = ngram_range, min_df = min_df)
    X = vectorizer.fit_transform(cleaned_corpus)
    ngram_freq = vectorizer.vocabulary_

    # Se ordenanan los n-gramas de mayor a menor número de palabras
    sorted_ngrams = sorted(ngram_freq.keys(), 
                           key = lambda x: -len(x.split()))
    return sorted_ngrams

def save_stopword_ngrams_corpus(ngrams):
    """
    Función que guarda en un archivo JSON la lista de n-gramas formados únicamente por 'stopwords'.

    Args:
        ngrams (list of str): Lista de n-gramas que se desea almacenar.

    Returns:
        None: La función no retorna ningún valor. El resultado se guarda en el archivo 'stopword_ngrams.json'.

    """
    # Se guarda la lista de n-gramas de tipo stopword en un fichero json para ser reutilizado
    with open("stopword_ngrams.json", 'w') as f:
        json.dump(ngrams, f, indent = 2)

In [ ]:
def remove_stopword_ngrams_from_dfs(df, stopword_ngrams, max_ngram_len, name_df):
    """
    Función que elimina n-gramas compuestos únicamente por 'stopwords' de la columna 'text' de un DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame a limpiar.
        stopword_ngrams (list of str): Lista de n-gramas de tipo 'stopword' que deben eliminarse.
        max_ngram_len (int): Longitud máxima de los n-gramas considerados durante la limpieza.
        name_df (str): Nombre identificador del DataFrame.

    Returns:
        pandas.DataFrame: DataFrame con la columna 'text' limpia y sin n-gramas de tipo 'stopword'. 
        También se eliminan las filas con texto vacío resultante tras el proceso.

    Notes:
        - El proceso recorre cada texto tokenizado, evaluando posibles coincidencias con los n-gramas proporcionados, desde el más largo al más corto.
        - Cuando se detecta un n-grama en el texto, se eliminan todos los tokens que lo componen.
        - Se utiliza la función `remove_empty_text_rows` para descartar filas que quedan vacías tras la limpieza.
    """
    # Se define una sub-función para limpiar cada texto
    def clean_text(text):
        # Se tokeniza el texto
        tokens = text.split()

        # Se definen las variables auxiliares
        keep = [True] * len(tokens)
        i = 0

        # Mientras no se hayan estudiado toda la frase al completo
        while i < len(tokens):
            matched = False
            # Se recorre desde el n-grama más largo hasta los bigramas
            for n in range(min(max_ngram_len, len(tokens) - i), 1, -1):
                # Se define el n-grama a estudiar
                ngram = ' '.join(tokens[i:i+n])
                # Si este n-grama está contenido en lso n-gramas de tipo stopword,
                # se mantiene la frase que sigue después del n-grama
                if ngram in stopword_ngrams:
                    for j in range(i, i+n):
                        keep[j] = False
                    i += n
                    matched = True
                    break
            # Si no hay ningún match con los ngramas, seguimos estudiando el texto
            if not matched:
                i += 1

        # Se reconstruye el texto con las palabras que no son stopwords
        cleaned = ' '.join([tok for tok, keep_flag in zip(tokens, keep) if keep_flag])
        
        # Se devuelve el texto limpio
        return cleaned

    # Se aplica la limpieza a la columna de texto
    df['text'] = df['text'].astype(str).apply(clean_text)

    # Se eliminan las filas con texto vacío después de quitar los n-gramas de tipo stopword
    return remove_empty_text_rows(df, name_df)


In [ ]:
''' STOPWORDS UNITARIAS '''
def remove_stopwords_single_from_dfs(df, name_df):
    """
    Función que elimina las 'stopwords' individuales de la columna 'text' de un DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame a limpiar.
        name_df (str): Nombre identificador del DataFrame.

    Returns:
        pandas.DataFrame: DataFrame con la columna 'text' sin 'stopwords', y sin filas con texto vacío tras el proceso.

    Notes:
        - Las 'stopwords' se definen mediante la lista `stop_words` de NLTK.
        - Cada texto se tokeniza por espacios y se reconstruye excluyendo los tokens presentes en 'stopwords'.
        - Se llama a `remove_empty_text_rows` para eliminar filas que resulten vacías después de la limpieza.
    """
    # Se define una sub-función para limpiar cada texto
    def remove_stopwords(text):
        # Se tokeniza el texto
        tokens = text.split()

        # Se eliminan las stopwords de NLTK encontradas en el texto
        cleaned_tokens = [token for token in tokens if token not in stop_words]

        # Se devuelve el texto limpio
        return ' '.join(cleaned_tokens)

    # Se aplica la limpieza a la columna de texto 
    df.loc[:, 'text'] = df['text'].apply(remove_stopwords)

    # Se eliminan las filas con texto vacío después de quitar las stopwords
    return remove_empty_text_rows(df, name_df)

### Generación del dataframe MELD para test

In [ ]:
def get_meld_test(df_train, test_size):
    """
    Función que genera o carga un conjunto de test a partir del dataset MELD, utilizando un `train_test_split` persistente en disco.

    Args:
        df_train (pandas.DataFrame): DataFrame original de entrenamiento del dataset MELD.
        test_size (float): Proporción del dataset que se desea utilizar como conjunto de prueba (entre 0 y 1).

    Returns:
        tuple:
            pandas.DataFrame: Subconjunto de entrenamiento tras la partición.
            pandas.DataFrame: Subconjunto de prueba.

    Notes:
        - Si existen los archivos `initial_MELD_train.csv` y `initial_MELD_test.csv`, se cargan directamente desde disco.
        - En caso contrario, se realiza la partición mediante `train_test_split` de scikit-learn y se guardan ambos subconjuntos en disco para uso futuro.
    """
    train_path = "initial_MELD_train.csv"
    test_path = "initial_MELD_test.csv"

    if os.path.exists(train_path) and os.path.exists(test_path):
        print("Cargando splits ya existentes...")
        # Se cargan los splits ya realizados previamente            
        df_train_split = pd.read_csv(train_path)
        df_test = pd.read_csv(test_path)
    else:
        print("Se generarán y guardarán los splits...")
        # Se realiza un train-test split para conseguir un dataframe de test para MELD
        df_train_split, df_test = train_test_split(df_train, test_size = test_size, random_state = 42)
    
        # Guardamos ambos dataset en ficheros CSV
        df_train_split.to_csv(train_path, index = False)
        df_test.to_csv(test_path, index = False)
    
    # Se devuleven los dataframes resultantes
    return df_train_split, df_test

In [ ]:
def filter_short_texts(df, name_df, min_words = 5):
    """
    Función que elimina del DataFrame las filas cuya columna 'text' contenga menos de un número mínimo de palabras.

    Args:
        df (pandas.DataFrame): DataFrame a evaluar.
        name_df (str): Nombre identificador del DataFrame
        min_words (int, optional): Número mínimo de palabras requerido para conservar una fila. Por defecto es 5.

    Returns:
        pandas.DataFrame: DataFrame filtrado, sin filas cuyo texto tenga menos de `min_words` palabras.

    Notes:
        - Esta función es útil para eliminar muestras consideradas ruidosas o poco informativas en tareas de modelaje.
        - Imprime por consola el número de filas antes y después del filtrado.
    """
    # Se calcula el número de filas original del dataframe
    original_len = len(df)
    
    # Se eliminan las muestras que tengan menos de 'min_words' palabras por generar ruido en las predicciones
    df = df[df['text'].apply(lambda x: len(str(x).split()) >= min_words)].reset_index()

    # Se calcula el número de filas final del dataframe y se muestra el mensaje
    final_len = len(df)
    print(f"{name_df} pasa de {original_len} filas a tener {final_len} filas.")
    return df

#### Feature Engineering

In [ ]:
def add_polarity_and_subjectivity(df):
    """
    Función que añade al DataFrame dos nuevas columnas con las métricas de polaridad y subjetividad calculadas a partir de la columna 'text'.

    Args:
        df (pandas.DataFrame): DataFrame a analizar.

    Returns:
        pandas.DataFrame: DataFrame con dos columnas adicionales:
            - 'polarity': Valor de polaridad del texto (entre -1.0 y 1.0).
            - 'subjectivity': Valor de subjetividad del texto (entre 0.0 y 1.0).

    Notes:
        - Se utiliza la librería `TextBlob` para el cálculo.
        - La polaridad indica cuán positivo o negativo es un texto.
        - La subjetividad mide el grado de opinión personal o juicio (más cercano a 1.0) frente a hechos objetivos (más cercano a 0.0).
    """
    # Se deninen las variables base
    polarities = []
    subjectivities = []

    # Por cada texto, se define su polaridad y su subjetividad
    for text in df['text']:
        blob = TextBlob(str(text))
        polarities.append(blob.sentiment.polarity)
        subjectivities.append(blob.sentiment.subjectivity)

    # Se añaden ambas variables como columnas del dataframe
    df['polarity'] = polarities
    df['subjectivity'] = subjectivities
    return df

In [ ]:
def feature_engineering_MELD(df):
    """
    Función que genera y normaliza variables derivadas para el dataset MELD para el posterior modelaje.

    Args:
        df (pandas.DataFrame): DataFrame del dataset MELD que contiene las columnas necesarias para el cálculo de nuevas características.

    Returns:
        pandas.DataFrame: DataFrame extendido con nuevas columnas.

    Notes:
        Las siguientes variables son generadas o transformadas:
        - 'duration': Diferencia entre 'EndTime_sec' y 'StartTime_sec', indicando la duración del clip.
        - 'turn_position': Posición relativa del turno dentro del diálogo, normalizada.
        - 'dialogue_id_norm': Valor de 'Dialogue_ID' normalizado.
        - 'utterance_id_norm': Valor de 'Utterance_ID' normalizado.
        - 'season_norm': Valor de 'Season' normalizado.
        - 'polarity' y 'subjectivity': Métricas lingüísticas extraídas mediante `TextBlob`.

        La función opera sobre una copia del DataFrame original para evitar efectos colaterales.
    """
    df = df.copy()

    # Se define las variables:
    # 'duration' como la duración del texto al ser hablado
    df['duration'] = df['EndTime_sec'] - df['StartTime_sec']

    # 'turn_position' como la posición del turno normalizada en el diálogo
    df['turn_position'] = df['Utterance_ID'] / df.groupby('Dialogue_ID')['Utterance_ID'].transform('max')

    # Normalización de las variables 'Dialogue_ID', 'Utterance_ID' y 'Season'
    df['dialogue_id_norm'] = df['Dialogue_ID'] / df['Dialogue_ID'].max()
    df['utterance_id_norm'] = df['Utterance_ID'] / df['Utterance_ID'].max()
    df['season_norm'] = df['Season'] / df['Season'].max()

    # 'polarity' y 'subjectivity'
    df = add_polarity_and_subjectivity(df)

    return df

In [ ]:
def feature_engineering_MOSEI(df):
    """
    Función que genera y normaliza variables derivadas para el dataset MOSEI para el posterior modelaje.

    Args:
        df (pandas.DataFrame): DataFrame del dataset MOSEI con las columnas necesarias para el cálculo de nuevas características.

    Returns:
        pandas.DataFrame: DataFrame extendido con nuevas columnas.

    Notes:
        Las siguientes variables son generadas o transformadas:
        - 'duration': Diferencia entre 'end_time' y 'start_time', indicando la duración del clip.
        - 'intensity': Suma total de la activación emocional entre todas las emociones.
        - 'dominant_emotion': Emoción con mayor valor en la fila.
        - 'positive_emotions': Número de emociones con valor positivo (> 0).
        - 'negative_emotions': Número de emociones con valor negativo (< 0).
        - 'polarity' y 'subjectivity': Métricas lingüísticas extraídas mediante `TextBlob`.

        La función opera sobre una copia del DataFrame original para evitar efectos colaterales.
    """
    df = df.copy()

    # Se define las variables:
    # 'duration' como la duración del texto al ser hablado
    df['duration'] = df['end_time'] - df['start_time']

    # 'intensity' como la intensidad emocional total de sumar todas las emociones
    emotion_cols = ['happy', 'sad', 'anger', 'surprise', 'disgust', 'fear', 'neutral']
    df['intensity'] = df[emotion_cols].sum(axis = 1)

    # 'dominant_emotion' como el ID de la primera emoción con mayor valor
    df['dominant_emotion'] = df[emotion_cols].idxmax(axis = 1)

    # Se define el diccionario de emociones de MOSEI
    emotions = ['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    emotion_dict_MOSEI = {emotion: i for i, emotion in enumerate(emotions)}

    # Se reemplazan los valores categóricos de la columna 'dominant_emotion' por valores numéricos
    df['dominant_emotion'] = df['dominant_emotion'].map(emotion_dict_MOSEI)

    # 'positive_emotions' como el número de emociones con valor positivo
    df['positive_emotions'] = df[emotion_cols].gt(0).sum(axis = 1)

    # 'positive_emotions' como el número de emociones con valor positivo
    df['negative_emotions'] = df[emotion_cols].lt(0).sum(axis = 1)

    # 'polarity' y 'subjectivity'
    df = add_polarity_and_subjectivity(df)

    return df

### Preprocesamiento del audio

#### Espectrogramas de Mel

In [ ]:
def create_mel_spectrograms_ffmpeg(df, output_folder, sr = 16000, n_mels = 128, fixed_duration = 4.0, target_shape = (224, 224)):
    """
    Función que genera espectrogramas de Mel a partir de archivos de audio o video usando `ffmpeg` y `librosa`, 
    y los guarda como archivos `.npy`.

    Args:
        df (pandas.DataFrame): DataFrame que contiene la columna 'video_path' y, en el caso de MOSEI, también 'start_time', 'end_time' y 'audio_name'.
        output_folder (str): Carpeta donde se guardarán los archivos `.npy` con los espectrogramas.
        sr (int, optional): Frecuencia de muestreo objetivo en Hz. Por defecto es 16000.
        n_mels (int, optional): Número de bandas Mel que se utilizarán. Por defecto es 128.
        fixed_duration (float, optional): Duración fija del audio en segundos para normalizar la longitud de entrada. Por defecto es 4.0 segundos.
        target_shape (tuple of int, optional): Dimensiones objetivo del espectrograma (alto, ancho). Por defecto es (224, 224).

    Returns:
        pandas.DataFrame: DataFrame original con una nueva columna 'mel_path', que contiene la ruta a cada espectrograma generado.

    Notes:
        - Para registros MOSEI, se recorta el fragmento de audio entre 'start_time' y 'end_time' usando `ffmpeg`.
        - Para MELD, se procesa directamente el archivo completo.
        - Los audios cortos se rellenan con ceros y los audios largos se recortan a `fixed_duration`.
        - Se aplica un escalado para ajustar el tamaño del espectrograma a `target_shape` mediante interpolación con `scipy.ndimage.zoom`.
        - Los espectrogramas se guardan como archivos `.npy` y se almacenan sus rutas en la columna 'mel_path'.
        - En caso de error o audio vacío, se imprime una advertencia y se asigna `None` en la columna correspondiente.
    """
    # Se crea el path final de los espectrogramas si no existe
    os.makedirs(output_folder, exist_ok = True)

    # Se define la lista de paths para los espectrogramas de Mel
    mel_paths = []

    # Por cada video o audio registrado en el dataframe
    for idx, row in tqdm(df.iterrows(), total = len(df)):
        # Extraemos el path del video o audio
        video_path = row['video_path']
        
        if 'audio_name' in df.columns:
            file_id = row['audio_name']
        else:
            file_id = os.path.splitext(os.path.basename(video_path))[0]


        # Definimos el path final del espetrograma como la carpeta del dataset y el nombre del video o audio
        output_path = os.path.join(output_folder, f"{file_id}.npy")

        try:
            if 'audio_name' in df.columns:
                # Caso MOSEI: se carga solo el fragmento necesario del audio original
                start_time = row['start_time']
                end_time = row['end_time']

                # Se extrae el audio entre start_time y end_time
                out, _ = (
                    ffmpeg
                    .input(video_path, ss = start_time, t = (end_time - start_time))
                    .output('pipe:', format = 'wav', ac = 1, ar = sr)
                    .run(capture_stdout=True, capture_stderr=True)
                )
            else:
                # Caso MELD: se carga todo el audio del video al estar previamente cortado
                out, _ = (
                    ffmpeg
                    .input(video_path)
                    .output('pipe:', format = 'wav', ac = 1, ar = sr)
                    .run(capture_stdout=True, capture_stderr=True)
                )

            # Se intenta cargar el audio extraido
            y, _ = librosa.load(io.BytesIO(out), sr = sr)

            # En el caso de que el audio esté corrupto o vacío
            if y is None or len(y) == 0:
                # No añadimos ningún path y saltamos a la siguiente pista de audio
                print(f"Audio vacío en {video_path}. Saltando.")
                mel_paths.append(None)
                continue

            # En el caso de que sí haya audio, este se ajusta al fixed_duration establecido
            # Primero, se calcula el largo esperado
            target_length = int(sr * fixed_duration)

            # Si la pista de audio es más pequeña que el target
            if len(y) < target_length:
                # Se rellena con ceros el tramo faltante, simulando silencio
                padding = target_length - len(y)
                y = np.pad(y, (0, padding), mode = 'constant')
            else:
                # Si es más largo que lo fijado, se recorta
                y = y[:target_length]

            # Se crea el Espectrograma de Mel
            S = librosa.feature.melspectrogram(y = y, sr = sr, n_mels = n_mels)
            S_db = librosa.power_to_db(S, ref = np.max)

            # Se ajusta el tamaño del espectrograma al tamaño común establecido target_shape
            zoom_factors = (target_shape[0] / S_db.shape[0], target_shape[1] / S_db.shape[1])
            S_db_resized = zoom(S_db, zoom_factors)

            # Se guarda el Espectrograma de Mel y se añade el path de este a la lista
            np.save(output_path, S_db_resized)
            mel_paths.append(output_path)

        except Exception as e:
            # En el caso de algún error al procesar los espectrogramas de Mel, se muestra por pantalla
            print(f"Error procesando {video_path}: {e}")
            mel_paths.append(None)

    # Se crea la columna mel_path con los paths a los respectivos Espectrogramas de Mel de cada video o audio
    df['mel_path'] = mel_paths

    # Se devuelve el dataframe actualizado
    return df

In [ ]:
def load_df_or_create_mel_spectrograms(df_audio, output_folder, csv_path):
    """
    Función que carga un DataFrame con rutas a espectrogramas de Mel si el archivo CSV ya existe, o los genera desde cero en caso contrario.

    Args:
        df_audio (pandas.DataFrame): DataFrame que contiene la columna 'video_path' (y opcionalmente 'start_time', 'end_time', 'audio_name' en MOSEI) usada para generar los espectrogramas.
        output_folder (str): Carpeta de destino donde se guardarán los archivos `.npy` con los espectrogramas de Mel.
        csv_path (str): Ruta relativa al CSV donde se almacenará o leerá el DataFrame resultante.

    Returns:
        pandas.DataFrame: DataFrame que contiene las rutas válidas a los espectrogramas de Mel generados para cada entrada de audio o video.

    Notes:
        - Si el archivo CSV especificado ya existe, se carga directamente.
        - Si no existe, se genera cada espectrograma mediante `create_mel_spectrograms_ffmpeg`, y se eliminan las filas sin espectrograma válido.
        - Se guarda un nuevo CSV con el resultado para evitar procesamiento redundante en futuras ejecuciones.
        - Imprime por consola información sobre la forma inicial y final del DataFrame procesado.
    """
    # Se define el path necesario para la carga del dataframe
    main_folder = '/kaggle/working/'
    file_path = os.path.join(main_folder, csv_path)
    
    if os.path.exists(file_path):
        # Si el CSV con los paths a los Espectrogramas de Mel existe, se carga el dataframe
        print(f"Cargando CSV procesado desde {file_path}...")
        df_mel = pd.read_csv(file_path)
    else:
        # Se define el shape original antes de generar los Espectrogramas de Mel
        original_shape = df_audio.shape

        # Si el CSV con los paths a los Espectrogramas de Mel NO existe, se generan los Espectrogramas
        print(f"Generando espectrogramas...")
        df_mel = create_mel_spectrograms_ffmpeg(df_audio, output_folder = output_folder)

        # Se eliminan las filas donde mel_path es None (o hay espetrograma por audio corrupto o vacío)
        df_mel = df_mel[~df_mel['mel_path'].isna()]
        df_mel = df_mel[df_mel['mel_path'].apply(lambda x: os.path.exists(x))]

        # Se muestra la comparativa de shapes
        print(f"{csv_path} -> Shape inicial: {original_shape} | Shape final: {df_mel.shape}")

        # Se guarda el dataframe que contiene los paths a los Espectrogramas
        df_mel.to_csv(csv_path, index = False)
        
    # Se devuelve el dataframe buscado
    return df_mel

In [ ]:
def process_in_chunks(df_audio, output_folder, final_csv_path, chunk_size = 5000):
    """
    Función que genera espectrogramas de Mel en fragmentos del DataFrame para evitar sobrecarga de memoria, y guarda un CSV final con todos los resultados.

    Args:
        df_audio (pandas.DataFrame): DataFrame que contiene las rutas a los archivos de audio o video a procesar.
        output_folder (str): Carpeta donde se guardarán los espectrogramas generados por cada fragmento.
        final_csv_path (str): Ruta del archivo CSV donde se almacenará el DataFrame final con los espectrogramas generados.
        chunk_size (int, optional): Número de filas por fragmento. Por defecto es 5000.

    Returns:
        pandas.DataFrame: DataFrame final que contiene todos los registros con sus correspondientes rutas a los espectrogramas de Mel.

    Notes:
        - El DataFrame se divide en fragmentos del tamaño definido por `chunk_size`.
        - Cada fragmento se procesa mediante `create_mel_spectrograms_ffmpeg`, generando los espectrogramas en su propia subcarpeta.
        - Se eliminan los registros con errores o sin espectrogramas válidos.
        - Cada fragmento se guarda como un CSV temporal, que luego se concatena para formar el DataFrame final.
        - El archivo CSV final se guarda en la ruta `final_csv_path` y contiene únicamente registros válidos.
        - Esta función fue cerada específicamente para el Dataframe del dataset MOSEI de entrenamiento debido a su gran peso contra el Kernel.
    """
    # Se crea la carpeta final si no existe
    os.makedirs(output_folder, exist_ok = True)

    # Se subdivide el dataframe en trozos para ir creando los espectrogramas
    chunks = [df_audio[i:i+chunk_size] for i in range(0, df_audio.shape[0], chunk_size)]
    all_chunk_paths = []

    # Por cada trozo del dataframe
    for idx, chunk in enumerate(chunks):
        print(f"\nProcesando trozo {idx+1}/{len(chunks)}... ({chunk.shape[0]} filas)")

        # Se define el nombre del path del trozo actual
        chunk_csv = f"chunk_{idx}.csv"
        chunk_folder = os.path.join(output_folder, f"chunk_{idx}")

        # Se crean los espectrogramas del trozo a estudiar
        df_chunk = create_mel_spectrograms_ffmpeg(chunk, output_folder=chunk_folder)

        # En caso de algún fallo, se eliminan dichos registros
        df_chunk = df_chunk[~df_chunk['mel_path'].isna()]
        df_chunk = df_chunk[df_chunk['mel_path'].apply(lambda x: os.path.exists(x))]

        # Se guarda el trozo procesado para no perder el progreso
        df_chunk.to_csv(chunk_csv, index = False)
        all_chunk_paths.append(chunk_csv)

    # Se concatenan todos los CSVs de cada trozo
    dfs = [pd.read_csv(f) for f in all_chunk_paths]
    final_df = pd.concat(dfs, ignore_index = True)

    # Se guarda el CSV final
    final_df.to_csv(final_csv_path, index = False)
    print(f"Dataset final guardado en {final_csv_path} ({final_df.shape[0]} filas)")

    # Se devuleve el dataframe final con sus paths de mel
    return final_df


## Funciones auxiliares para el Entrenamiento

### Data Augmentation

In [ ]:
def random_swap(words, n = 2):
    """
    Función que realiza intercambios aleatorios de posición entre palabras de una lista.

    Args:
        words (list of str): Lista de palabras sobre la que se aplicarán los intercambios.
        n (int, optional): Número de intercambios aleatorios que se desea realizar. Por defecto es 2.

    Returns:
        list of str: Nueva lista de palabras con los elementos intercambiados aleatoriamente.

    Notes:
        - Los índices de las palabras a intercambiar se eligen de forma aleatoria e independiente.
        - La función no modifica la lista original, sino que trabaja sobre una copia.
    """
    # Se crea una copia de la lista de palabras del texto
    new_words = words.copy()
    
    # Se intercambian aleatoriamente las palabras de posición
    for _ in range(n):
        idx1 = random.randint(0, len(new_words)-1)
        idx2 = random.randint(0, len(new_words)-1)
        new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
        
    return new_words

def simple_text_augmentation(text, n_swaps = 2):
    """
    Función que aplica una técnica simple de aumento de datos textuales mediante el intercambio aleatorio de palabras.

    Args:
        text (str): Cadena de texto sobre la que se desea aplicar el aumento de datos.
        n_swaps (int, optional): Número de pares de palabras que se intercambiarán aleatoriamente. Por defecto es 2.

    Returns:
        str: Nueva cadena de texto resultante tras realizar los intercambios de palabras.

    Notes:
        - La función `random_swap` se encarga de realizar los intercambios aleatorios.
    """
    # Se genera la lista de palabras del texto
    words = text.split()

    # Se intercambian las posiciones de ciertas palabras del texto aleatoriamente
    new_words = random_swap(words, n = n_swaps)

    # Se devuelve el nuevo texto para el data augmentation
    return ' '.join(new_words)

def augment_dataframe_balanced(df):
    """
    Función que equilibra un DataFrame aplicando aumento de datos textual para las clases minoritarias en la variable 'sentiment'.

    Args:
        df (pandas.DataFrame): DataFrame original.

    Returns:
        pandas.DataFrame: DataFrame balanceado, con las clases igualadas en número de muestras mediante aumento sintético de datos.

    Notes:
        - Se identifican las clases minoritarias en 'sentiment' y se calcula cuántas muestras faltan para igualar a la clase mayoritaria.
        - Se realiza muestreo con reemplazo sobre las clases minoritarias y se aplica `simple_text_augmentation` para generar textos aumentados.
    """
    # Se calcula el número de filas original del dataframe
    original_len = len(df)

    # Se define la lista de dataframes aumentados
    augmented_dfs = []

    # Se calcula el número de muestras por clase de sentimiento
    class_counts = df['sentiment'].value_counts()

    # Se extrae el máximo número de muestras que hay en una de las clases 
    max_count = class_counts.max()

    # Por cada etiqueta y recuento de muestras de los sentimientos
    for label, count in class_counts.items():
        # Se extrae el sub-dataframe de dicho sentimiento
        df_label = df[df['sentiment'] == label]
        
        # Se calcula cuantasmuestras son necesarias para igualar la clase minoritaria a la clase mayoritaria
        n_needed = max_count - count

        # Si se encuentra una clase minoritaria
        if n_needed > 0:
            # Se aplica el data augmentation por cambios posionales para obtener las muestras necesarias
            sampled_df = df_label.sample(n = n_needed, replace = True, random_state = 42)
            augmented_texts = [simple_text_augmentation(text) for text in sampled_df['text']]
            augmented_df = sampled_df.copy()
            augmented_df['text'] = augmented_texts
            augmented_dfs.append(augmented_df)

    # Se concatenan el dataframe original y lso sub-dataframes del data augmentation
    df_augmented = pd.concat([df] + augmented_dfs, ignore_index = True)

    # Se calcula el número de filas final del dataframe
    final_len = len(df_augmented)

    # Se calcula el número final de muestras por clase de sentimiento
    final_class_counts = df_augmented['sentiment'].value_counts()

    # Se muestra el balanceo de las muestras
    print("\n Resultado del balanceo:")
    print(f"- Tamaño original del dataset: {original_len}")
    print(f"- Tamaño después del augmentación: {final_len}")
    print(f"- Número de muestras por clase antes del balanceo: {class_counts.sort_index()}")
    print(f"- Número de muestras por clase después del balanceo: {final_class_counts.sort_index()}")

    # Se devuelve el dataframe balanceado y aumentado
    return df_augmented

### Entrenamiento en texto

In [ ]:
def prepare_text_feature_label_data(df, feature_cols):
    """
    Función que extrae y organiza los datos de texto, labels y características numéricas desde un DataFrame para su uso en modelaje.

    Args:
        df (pandas.DataFrame): DataFrame que contiene las columnas 'text', 'sentiment' y las características especificadas.
        feature_cols (list of str): Lista de nombres de columnas numéricas que se utilizarán como variables adicionales (features).

    Returns:
        tuple:
            list of str: Lista de textos provenientes de la columna 'text'.
            list of int: Lista de labels de sentimiento convertidas a enteros.
            numpy.ndarray: Matriz de características numéricas extraídas de las columnas especificadas.
    """
    # Se extrae la variable de texto a formato lista
    text = df['text'].tolist()

    # Se extraen las etiquetas del sentimiento a formato lista
    labels = df['sentiment'].astype(int).tolist()

    # Se definen las features
    features = df[feature_cols].values
    return text, labels, features

In [ ]:
def measure_max_len(tokenizer, list_datasets, name_dataset, name_tokenizer, add_special_tokens = True):
    """
    Función que determina la longitud máxima de tokens requerida para procesar un conjunto de textos, sin exceder el límite del tokenizador.

    Args:
        tokenizer (transformers.PreTrainedTokenizer): Tokenizador que se utilizará para codificar los textos.
        list_datasets (list of list of str): Lista de listas de textos a evaluar.
        name_dataset (str): Nombre del dataset.
        name_tokenizer (str): Nombre del tokenizador.
        add_special_tokens (bool, optional): Indica si se deben incluir tokens especiales (CLS, SEP, etc.) en la codificación. Por defecto es True.

    Returns:
        int: Longitud máxima de tokens a utilizar, ajustada al límite del modelo (`tokenizer.model_max_length`).

    """
    # Se define la variable base
    max_len = 0

    # Entre todos los datasets, se buscam la longitud de texto máxima que el modelo procesará
    for text in np.hstack(list_datasets):
        input_ids = tokenizer.encode(text, add_special_tokens = add_special_tokens)
        # Se actualiza la variable base
        max_len = max(max_len, len(input_ids))
    
    print(f'Max token length for {name_dataset}: {max_len}')
    print(f'Model {name_tokenizer} max token length: {tokenizer.model_max_length}')

    # Se devuelve el valor mínimo entre el máximo del tokenizados y el máximo encontrado
    return min(tokenizer.model_max_length, max_len)

In [ ]:
def plot_training_history(history):
    """
    Función que genera gráficos de la evolución del entrenamiento de un modelo, incluyendo Loss, F1-score y Accuracy por epoch.

    Args:
        history (dict): Diccionario que contiene las métricas por epoch. Debe incluir:
            - 'train_loss', 'val_loss': Loss en entrenamiento y validación.
            - 'train_f1', 'val_f1': F1-score en entrenamiento y validación.
            - 'train_acc', 'val_acc': Accuracy en entrenamiento y validación.

    Returns:
        None: La función no devuelve valores; muestra directamente los gráficos en pantalla.

    Notes:
        - Se crean tres gráficos en una sola figura: uno para cada métrica.
        - Cada gráfico compara el rendimiento en entrenamiento y validación a lo largo de las epochs.
    """
    # Se definen el número de epochs totales
    epochs = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize = (18,5))

    # Se plotea la loss
    plt.subplot(1, 3, 1)
    plt.plot(epochs, history['train_loss'], label = 'Train Loss')
    plt.plot(epochs, history['val_loss'], label = 'Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss over Epochs')
    plt.legend()

    # Se plotea el F1-score
    plt.subplot(1, 3, 2)
    plt.plot(epochs, history['train_f1'], label = 'Train F1')
    plt.plot(epochs, history['val_f1'], label = 'Validation F1')
    plt.xlabel('Epochs')
    plt.ylabel('F1-score')
    plt.title('F1-score over Epochs')
    plt.legend()

    # Se plotea el Accuracy
    plt.subplot(1, 3, 3)
    plt.plot(epochs, history['train_acc'], label = 'Train Accuracy')
    plt.plot(epochs, history['val_acc'], label = 'Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()

    plt.show()

In [ ]:
def grid_search_hyperparameters(model_name, tokenizer, texts_train, features_train, labels_train, texts_dev, features_dev, labels_dev, max_len):
    """
    Función que realiza un 'grid search' sobre ciertas combinaciones de hiperparámetros para entrenar un modelo Transformer.

    Args:
        model_name (str): Nombre del modelo Transformer a utilizar.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizador correspondiente al modelo.
        texts_train (list of str): Textos del conjunto de entrenamiento.
        features_train (numpy.ndarray): Variables numéricas del conjunto de entrenamiento.
        labels_train (list or numpy.ndarray): Etiquetas del conjunto de entrenamiento.
        texts_dev (list of str): Textos del conjunto de validación.
        features_dev (numpy.ndarray): Variables numéricas del conjunto de validación.
        labels_dev (list or numpy.ndarray): Etiquetas del conjunto de validación.
        max_len (int): Longitud máxima de los textos tras tokenización.

    Returns:
        dict: Diccionario con la mejor combinación de hiperparámetros encontrada, incluyendo:
            - 'batch_size'
            - 'epochs'
            - 'learning_rate'
            - 'warm_up'
            - 'best_f1'

    Notes:
        - Se evalúan combinaciones de `batch_size`, `epochs` y `learning_rate`.
        - Para cada combinación, se entrena un modelo `HybridTransformerClassifier` y se evalúa el F1-score sobre el conjunto de validación.
        - El entrenamiento se realiza utilizando `train_and_evaluate` y se almacena la mejor puntuación.
        - El resultado final es la configuración con mayor F1-score.
        - La búsqueda utiliza el dispositivo CUDA por defecto.
    """
    # Se define el grid de hiperparámetros a probar
    batch_sizes = [32, 64]
    epochs_list = [8, 10]
    learning_rates = [2e-5, 1e-5]

    # Se define la lista para los resultados y el device
    results = []
    device = 'cuda'

    # Por cada combianción
    for batch_size, epochs, lr in product(batch_sizes, epochs_list, learning_rates):
        print(f"\n Combinación a probar: batch = {batch_size}, epochs = {epochs}, LR = {lr}")
        
        # Se crean los datasets y los dataloaders
        train_dataset = SentimentDataset(texts_train, features_train, labels_train, tokenizer, max_len)
        dev_dataset = SentimentDataset(texts_dev, features_dev, labels_dev, tokenizer, max_len)

        train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
        dev_loader = DataLoader(dev_dataset, batch_size = batch_size)

        # Se define el modelo a estrenar
        model = HybridTransformerClassifier(model_name, num_features = features_train.shape[1])

        # Se entrena el modelo con la combinación escogida
        trained_model, best_f1 = train_and_evaluate(
            model,
            train_loader,
            dev_loader,
            labels_train,
            device,
            epochs = epochs,
            lr = lr,
            warm_up = 0.1,
            patience = 2,
            verbose = False
        )

        # Se guardan los resultados del entrenamiento
        results.append({
            "batch_size": batch_size,
            "epochs": epochs,
            "learning_rate": lr,
            "warm_up": 0.1,
            "best_f1": best_f1
        })

    # Se ordenan los resultados de mejor a peor segun su F1-score
    results = sorted(results, key=lambda x: x["best_f1"], reverse=True)

    # Se muestra la mejor combinación encontrada
    print("\nMejor combinaión:")
    print(f"F1-score = {results[0]['best_f1']:.4f} | batch = {results[0]['batch_size']}, epochs = {results[0]['epochs']}, LR = {results[0]['learning_rate']}")

    # Se devuelven los mejores hiperparámetros
    return results[0]

In [ ]:
def plot_feature_importance(features, labels, feature_names, top_n = 9, random_state = 42):
    """
    Función que entrena un clasificador Random Forest para calcular la importancia de las variables numéricas 
    y genera un gráfico con las características más relevantes.

    Args:
        features (numpy.ndarray or pandas.DataFrame): Variables predictoras.
        labels (array-like): Vector de etiquetas objetivo para el entrenamiento del modelo.
        feature_names (list of str): Lista de nombres de las variables que corresponden a las columnas de `features`.
        top_n (int, optional): Número de características más importantes a visualizar. Por defecto es 9.
        random_state (int, optional): Semilla utilizada para reproducibilidad del Random Forest. Por defecto es 42.

    Returns:
        None: La función no retorna nada; muestra un gráfico de barras horizontales con las características más importantes.

    Notes:
        - Se utiliza un `RandomForestClassifier` con 100 árboles para calcular la importancia de cada variable.
        - Las importancias se ordenan de mayor a menor.
    """
    # Se asegura primero que features es un array
    if isinstance(features, pd.DataFrame):
        X = features.values
    else:
        X = features

    # Se establecen las etiquetas como y
    y = labels

    # Se entrena el RandomForest para sacar la feature importance
    rf = RandomForestClassifier(n_estimators = 100, 
                                random_state = random_state)
    rf.fit(X, y)

    # Se ebtienen las importancias y se ordenan de más a menos importantes
    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Se selecciona el top n features
    top_indices = indices[:top_n]

    # Se pletean las importanias
    plt.figure(figsize = (10, 6))
    plt.barh(range(top_n), importances[top_indices][::-1], align = 'center')
    plt.yticks(range(top_n), [feature_names[i] for i in top_indices][::-1])
    plt.xlabel('Feature Importance')
    plt.title('Top Feature Importances')
    plt.grid(True, linestyle='--', alpha = 0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
class SentimentDataset(Dataset):
    """
    Dataset personalizado para tareas de clasificación de sentimientos que combina texto tokenizado y variables numéricas adicionales.

    Args:
        texts (list of str): Lista de textos que se desean tokenizar.
        features (numpy.ndarray or list): Matriz de características numéricas asociadas a cada texto.
        labels (list or numpy.ndarray): Etiquetas de clase para cada muestra.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizador utilizado para procesar los textos.
        max_len (int, optional): Longitud máxima permitida para los textos tokenizados. Por defecto es 128.

    Methods:
        __len__(): Devuelve el número total de muestras del dataset.
        __getitem__(idx): Devuelve un diccionario con los tensores necesarios para el modelo:
            - 'input_ids': IDs de los tokens.
            - 'attention_mask': Máscara de atención para el modelo.
            - 'features': Variables numéricas adicionales.
            - 'labels': Etiqueta de la muestra.
    """
    def __init__(self, texts, features, labels, tokenizer, max_len = 128):
        self.texts = texts
        self.features = features
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
         # Se devuleve la cantidad total de muestras
        return len(self.labels)

    def __getitem__(self, idx):
        # S extraen el texto, las features y la etiqueta del índice idx
        text = self.texts[idx]
        feature = self.features[idx]
        label = self.labels[idx]
        
        # Se tokeniza el texto correspondiente
        encoding = self.tokenizer(
            text,
            padding = 'max_length',
            truncation = True,
            max_length = self.max_len,
            return_tensors = 'pt'
        )

        # Se devuleven los tensores resultantes
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'features': torch.tensor(feature, dtype = torch.float),
            'labels': torch.tensor(label, dtype = torch.long)
        }

In [ ]:
class HybridTransformerClassifier(nn.Module):
    """
    Modelo híbrido que combina embeddings de un modelo Transformer preentrenado con variables numéricas adicionales para realizar clasificación.

    Args:
        model_name (str): Nombre del modelo Transformer.
        num_features (int): Número de características numéricas adicionales.
        num_classes (int, optional): Número de clases para la clasificación. Por defecto es 3.

    Attributes:
        transformer (AutoModel): Modelo Transformer preentrenado cargado.
        feature_layer (nn.Linear): Capa lineal que transforma las variables numéricas a un espacio intermedio de dimensión 128.
        norm (nn.LayerNorm): Capa de normalización sobre la concatenación del embedding y las features.
        classifier (nn.Sequential): Red neuronal que realiza la clasificación final a partir del vector combinado.

    Forward Args:
        input_ids (torch.Tensor): Tensor de IDs de tokens con forma (batch_size, seq_len).
        attention_mask (torch.Tensor): Máscara de atención con forma (batch_size, seq_len).
        features (torch.Tensor): Tensor de variables numéricas adicionales con forma (batch_size, num_features).

    Returns:
        torch.Tensor: Logits de clase con forma (batch_size, num_classes).

    Notes:
        - Se utiliza `pooler_output` del Transformer como representación del texto. Si no está disponible, se usa el primer token de salida.
        - Las variables numéricas se proyectan y concatenan con el embedding textual antes de pasar por el clasificador.
    """
    def __init__(self, model_name: str, num_features: int, num_classes = 3):
        super().__init__()

        # Se define el transformer
        self.transformer = AutoModel.from_pretrained(model_name)

        # Se detecta automáticamente el tamaño del hidden del modelo
        hidden_size = self.transformer.config.hidden_size

        # Se define el procesador para las features
        self.feature_layer = nn.Linear(num_features, 128)

        # Se define una capa de normalización
        self.norm = nn.LayerNorm(hidden_size + 128)

        # Se define el clasificador
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size + 128, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, input_ids, attention_mask, features):
        # Se extrae el embedding del texto desde el pooler_output del transformer
        outputs = self.transformer(input_ids = input_ids, attention_mask = attention_mask)

        if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
            pooled_output = outputs.pooler_output
        else:
            # Para XLNet se usará el primer token al no tener pooler_output
            pooled_output = outputs.last_hidden_state[:, 0]

        # Se procesan de las features 
        feature_out = self.feature_layer(features)

        # Ae concatenan el texto y las features
        combined = torch.cat((pooled_output, feature_out), dim = 1)

        # Se clasifica
        logits = self.classifier(combined)
        return logits

In [ ]:
class EarlyStopping:
    """
    Clase que implementa la técnica de' Early Stopping' durante el entrenamiento de modelos para evitar sobreajuste.

    Args:
        patience (int, optional): Número de epochs consecutivas sin mejora del F1-score antes de detener el entrenamiento. Por defecto es 2.
        verbose (bool, optional): Indica si se deben mostrar mensajes informativos cuando se guarda un nuevo mejor modelo. Por defecto es True.
        save_path (str, optional): Ruta del archivo donde se guardará el mejor modelo. Por defecto es 'best_model.pt'.

    Attributes:
        best_score (float or None): Mejor valor de F1-score observado hasta el momento.
        counter (int): Contador de epochs consecutivas sin mejora.
        early_stop (bool): Indicador de si debe detenerse el entrenamiento.
        save_path (str): Ruta al archivo donde se guarda el mejor modelo.

    Methods:
        __call__(score, model): Evalúa si el nuevo score mejora al anterior; guarda el modelo si mejora o aumenta el contador si no.
        save_checkpoint(model): Guarda el estado del modelo en `save_path`.
    """
    # Se define la clase EarlyStopping
    def __init__(self, patience = 2, verbose = True, save_path = 'best_model.pt'):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.verbose = verbose
        self.save_path = save_path

    # Se define la llamada al earlyStopping cuando el entrenamiento se estanca
    def __call__(self, score, model):
        if self.best_score is None or score > self.best_score:
            self.best_score = score
            self.counter = 0
            self.save_checkpoint(model)
            if self.verbose:
                print(f"New best model saved with F1: {score:.4f}")
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                
     # Se define el guardado del mejor modelo enoctrado            
    def save_checkpoint(self, model):
        torch.save(model.state_dict(), self.save_path)

In [ ]:
def epoch_train(model, dataloader, optimizer, scheduler, loss_fn, device, verbose = True):
    """
    Función que entrena un modelo durante una epoch completa usando un conjunto de datos proporcionado por un DataLoader.

    Args:
        model (torch.nn.Module): Modelo que se desea entrenar.
        dataloader (torch.utils.data.DataLoader): DataLoader que proporciona los batches de entrenamiento.
        optimizer (torch.optim.Optimizer): Optimizador para actualizar los pesos del modelo.
        scheduler (torch.optim.lr_scheduler): Scheduler para ajustar la tasa de aprendizaje durante el entrenamiento.
        loss_fn (callable): Función de pérdida a optimizar.
        device (str): Dispositivo en el que se ejecutará el entrenamiento (es 'cuda').
        verbose (bool, optional): Si es True, se muestra una barra de progreso. Por defecto es True.

    Returns:
        tuple:
            float: Loss durante la epoch.
            float: Aaccuracy en la epoch.
            float: F1-score ponderado en la epoch.

    Notes:
        - Se pone el modelo en modo entrenamiento con `model.train()`.
        - Los gradientes se reinician en cada batch con `optimizer.zero_grad()`.
        - Se realiza forward pass, cálculo de pérdida, backward pass y actualización de pesos.
        - Se acumulan predicciones y etiquetas reales para calcular métricas al final de la época.
    """
    # Se pone el modelo en modo entreno
    model.train()

    # Se definen las variables base para la loss y las predicciones
    total_loss = 0
    all_preds, all_labels = [], []

    # Por cada batch
    for batch in tqdm(dataloader, disable = not verbose):
        # Se extrae la información necesaria del dataloader para enviarla al device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        features = batch['features'].to(device)
        labels = batch['labels'].to(device)

        # Se reinician los gradientes para evitar problemas de gradiente
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids = input_ids, 
                        attention_mask = attention_mask, 
                        features = features)
        
        # Se calcula la loss
        loss = loss_fn(outputs, labels)

        # Se realiza backpropagation y se actualizan los pesos y el scheduler
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Se acumula la loss y extraen las predicciones
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim = 1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

    # Se calculan las métricas globales: avg loss, ACC y F1-score
    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average = 'weighted')
    return avg_loss, acc, f1

In [ ]:
def evaluate(model, dataloader, loss_fn, device, verbose = True):
    """
    Función que evalúa un modelo sobre un conjunto de validación sin actualizar los pesos, y calcula métricas de rendimiento.

    Args:
        model (torch.nn.Module): Modelo a evaluar.
        dataloader (torch.utils.data.DataLoader): DataLoader que proporciona los batches de validación.
        loss_fn (callable): Función de pérdida utilizada para calcular la Loss en cada batch.
        device (str): Dispositivo en el que se ejecuta la evaluación (es 'cuda').
        verbose (bool, optional): Si es True, puede activarse una barra de progreso externa. Por defecto es True.

    Returns:
        tuple:
            float: Lloss durante la evaluación.
            float: Accuracy total.
            float: F1-score ponderado.
            dict: Diccionario con las métricas detalladas del classification report (por clase y globales).

    Notes:
        - Se desactiva el cálculo de gradientes mediante `torch.no_grad()` para ahorrar memoria y acelerar la evaluación.
        - Se pone el modelo en modo evaluación con `model.eval()`.
    """
    # Se pone el modelo en modo evaluación
    model.eval()

    # Se definen las variables base para la loss y las predicciones
    total_loss = 0
    all_preds, all_labels = [], []

    # Se le establece al modelo que no calcule o guarde los gradientes, así ahorrando memoria 
    # y aumentando la velocidad de las predicciones
    with torch.no_grad():
        # Por cada batch
        for batch in dataloader:
            # Se extrae la información necesaria del dataloader para enviarla al device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            features = batch['features'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass sin backpropagation
            outputs = model(input_ids = input_ids, 
                            attention_mask = attention_mask, 
                            features = features)
            
            # Se calcula la loss y se acumula
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            # Se extraen las predicciones
            preds = torch.argmax(outputs, dim = 1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Se calculan las métricas globales: avg loss, ACC, F1-score y classification report
    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average = 'weighted', zero_division = 0)
    report = classification_report(all_labels, all_preds, output_dict = True, zero_division = 0)
    return avg_loss, acc, f1, report

In [ ]:
def train_and_evaluate(model, train_dataloader, dev_dataloader, labels_train, device, epochs = 5, lr = 2e-5, patience = 5, freeze_epochs = 2, warm_up =  0.1, verbose = True):
    """
    Función que entrena un modelo durante varias épocas, evalúa su rendimiento en un conjunto de validación y aplica detención temprana (early stopping).

    Args:
        model (torch.nn.Module): Modelo a entrenar.
        train_dataloader (DataLoader): DataLoader con los datos de entrenamiento.
        dev_dataloader (DataLoader): DataLoader con los datos de validación.
        labels_train (array): Etiquetas del conjunto de entrenamiento, usadas para calcular pesos de clase.
        device (str): Dispositivo de cómputo (es 'cuda').
        epochs (int, optional): Número total de epochs de entrenamiento. Por defecto es 5.
        lr (float, optional): Tasa de aprendizaje inicial. Por defecto es 2e-5.
        patience (int, optional): Número de epochs sin mejora antes de detener el entrenamiento anticipadamente. Por defecto es 5.
        freeze_epochs (int, optional): Número de épocas iniciales durante las cuales las capas del Transformer permanecen congeladas. Por defecto es 2.
        warm_up (float, optional): Porcentaje de pasos de entrenamiento usados para calentamiento del learning rate. Por defecto es 0.1.
        verbose (bool, optional): Si es True, se muestra información de entrenamiento por consola. Por defecto es True.

    Returns:
        tuple:
            torch.nn.Module: Modelo con los pesos del mejor estado encontrado según F1-score en validación.
            float: Mejor F1-score alcanzado durante el mejor entrenamiento.

    Notes:
        - Se usa el optimizador AdamW y un scheduler con calentamiento y decaimiento tipo coseno.
        - Se calculan pesos de clase para la función de pérdida (`CrossEntropyLoss`) de manera equilibrada.
        - Las capas del Transformer pueden permanecer congeladas durante `freeze_epochs` iniciales.
        - Se aplica early stopping basado en el F1-score de validación.
    """
    # Se manda el modelo al device
    model.to(device)

    # Se defin el optimizador AdamW
    optimizer = AdamW(model.parameters(), lr = lr)

    # Se calculan los steps del dataloader
    total_steps = len(train_dataloader) * epochs
    warmup_steps = int(warm_up * total_steps)
    
    # Se define el scheduler para el LR
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_warmup_steps = warmup_steps,
                                                num_training_steps = total_steps)

    # Se definen los pesos de las clases a predecir
    labels = np.array(labels_train)
    
    # Se calculan los pesos de clas clases
    class_weights = compute_class_weight(class_weight = 'balanced', 
                                         classes = np.unique(labels), 
                                         y = labels)
    class_weights = torch.tensor(class_weights, dtype = torch.float)

    # Se define la función de loss con pesos
    loss_fn = nn.CrossEntropyLoss(weight = class_weights.to(device))

    # Se define el EalyStopping
    early_stopper = EarlyStopping(patience = patience)

    # Se definen las variables base
    best_f1 = 0
    best_model_state = None
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'train_f1': [], 'val_f1': []}

    # Se congelan las capas del transformer inicialmente
    for param in model.transformer.parameters():
        param.requires_grad = False

    # Por cada epoch
    for epoch in range(1, epochs + 1):
        if verbose:
            print(f"\n Epoch {epoch}")
        # Descongelar el transformer después de algunas épocas
        if epoch == freeze_epochs + 1:
            for param in model.transformer.parameters():
                param.requires_grad = True
                
        # Se entrena el modelo
        train_loss, train_acc, train_f1 = epoch_train(model, train_dataloader, optimizer, scheduler, loss_fn, device, verbose = verbose)
        if verbose:
            print(f"Train Loss: {train_loss:.4f}, ACC: {train_acc:.4f}, F1-score: {train_f1:.4f}")

        # Se valida el modelo
        val_loss, val_acc, val_f1, _ = evaluate(model, dev_dataloader, loss_fn, device, verbose = verbose)
        if verbose:
            print(f"Val Loss: {val_loss:.4f}, ACC: {val_acc:.4f}, F1-score: {val_f1:.4f}")

        # Se actualiza el LR en función de la loss de validación
        # scheduler.step(val_loss)

        # Si ´se encuentra un mejor modelo, se guarda
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_state = model.state_dict()
        
        # Se añaden lso resultados a la historia del entrenamiento
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)

        # Se llama al Early stopper para saber si hay que parar el entrenamiento
        early_stopper(val_f1, model)
        if early_stopper.early_stop:
            print("Early stopping triggered.")
            break

    # Se guardan el estado del mejor modelo encontrado entre las epochs
    model.load_state_dict(best_model_state)
    
    if verbose:
        # Se plotea la historia del entrenamiento
        plot_training_history(history)

    # Se devuleve le modelo y la mejor F1-score
    return model, best_f1

In [ ]:
def run_model_competition(competition_dict, device = 'cuda', epochs = 5, lr = 2e-5, patience = 2, freeze_epochs = 1, warm_up = 0.1, verbose = False):
    """
    Función que entrena y evalúa múltiples modelos en una competición para comparar su rendimiento sobre un conjunto de validación.

    Args:
        competition_dict (dict): Diccionario donde cada clave es el nombre del modelo y su valor es una tupla con:
            (modelo, dataloader de entrenamiento, dataloader de validación, etiquetas de entrenamiento).
        device (str, optional): Dispositivo sobre el cual se ejecutará el entrenamiento. Por defecto es 'cuda'.
        epochs (int, optional): Número de epochs de entrenamiento. Por defecto es 5.
        lr (float, optional): Tasa de aprendizaje inicial. Por defecto es 2e-5.
        patience (int, optional): Número de epochs sin mejora antes de aplicar early stopping. Por defecto es 2.
        freeze_epochs (int, optional): Número de epochs iniciales con el modelo Transformer congelado. Por defecto es 1.
        warm_up (float, optional): Proporción de pasos para el calentamiento del learning rate. Por defecto es 0.1.
        verbose (bool, optional): Si es True, se imprime información detallada durante el entrenamiento. Por defecto es False.

    Returns:
        tuple:
            str: Nombre del modelo con mejor F1-score macro.
            torch.nn.Module: Modelo entrenado correspondiente al mejor rendimiento.
            dict: Diccionario con las métricas del mejor modelo, incluyendo:
                - 'accuracy': Accuracy total.
                - 'f1_macro': F1-score macro.
                - 'f1_weighted': F1-score ponderado por clase.
                - 'aucs': Accuracy por clase si es posible.
                - 'model': Modelo entrenado.

    Notes:
        - Cada modelo se entrena mediante la función `train_and_evaluate`.
        - Se evalúa el rendimiento sobre el conjunto de validación usando `predict`.
        - Se calcula el AUC por clase cuando sea posible; si no, se asigna `NaN`.
        - El modelo con mayor F1-score macro es seleccionado como el mejor.
    """
    # Se define el diccionario de los resultados
    results = {}

    # Por cada candidato en la competición
    for model_name, (model, train_dataloader, dev_dataloader, labels_train) in competition_dict.items():
        print(f"Entrenando el modelo {model_name}")

        # Se manda el modelo a entrenar
        trained_model, _ = train_and_evaluate(model, 
                                              train_dataloader, 
                                              dev_dataloader, 
                                              labels_train = labels_train, 
                                              device = device,
                                              epochs = epochs,
                                              lr = lr,
                                              patience = patience,
                                              freeze_epochs = freeze_epochs,
                                              warm_up = warm_up,
                                              verbose = verbose)

        # Se predice usando el modelo entrenamos
        preds_dev, probs_dev = predict(trained_model, dev_dataloader, device)
        true_labels_dev = [batch['labels'].cpu().numpy() for batch in dev_dataloader]
        true_labels_dev = np.concatenate(true_labels_dev)

        # Se calculan las métricas pertinentes al estudio: Accuracy, F1-macro y F1-weighted
        acc = accuracy_score(true_labels_dev, preds_dev)
        f1_macro = f1_score(true_labels_dev, preds_dev, average = 'macro')
        f1_weighted = f1_score(true_labels_dev, preds_dev, average = 'weighted')

        # Se calcula el AUC
        aucs = {}
        for i in range(probs_dev.shape[1]):
            try:
                aucs[i] = roc_auc_score((true_labels_dev == i).astype(int), probs_dev[:, i])
            except:
                aucs[i] = np.nan

        # Se guardan los resultados del modelo entrenado con sus métricas
        results[model_name] = {
            'accuracy': acc,
            'f1_macro': f1_macro,
            'f1_weighted': f1_weighted,
            'aucs': aucs,
            'model': trained_model
        }

        print(f"{model_name} - ACC: {acc:.4f} - F1-macro: {f1_macro:.4f}\n")

    # Se escoge el mejor modelo basándose en F1-macro
    best_model_name = max(results, key = lambda x: results[x]['f1_macro'])
    best_model = results[best_model_name]['model']

    print(f"\nMejor modelo: {best_model_name} con F1-macro: {results[best_model_name]['f1_macro']:.4f}")

    # Se devuelve el mejor modelo y sus resultados
    return best_model_name, best_model, results[best_model_name]


In [ ]:
def add_predictions_to_csv(original_csv_path, output_csv_path, predictions):
    """
    Función que añade las predicciones del modelo como columnas a un CSV existente y guarda el resultado en una nueva ruta.

    Args:
        original_csv_path (str): Ruta al archivo CSV original.
        output_csv_path (str): Ruta donde se guardará el nuevo archivo CSV con las predicciones añadidas.
        predictions (numpy.ndarray): Matriz de predicciones con forma (n_samples, 3), correspondiente a las clases [negativo, neutral, positivo].

    Returns:
        None: La función guarda el archivo modificado, no devuelve ningún valor.
    """
    # Se carga el CSV original
    df = pd.read_csv(original_csv_path)

    # Se añaden las predicciones como nuevas columnas
    df['pred_negative'] = predictions[:, 0]
    df['pred_neutral'] = predictions[:, 1]
    df['pred_positive'] = predictions[:, 2]

    # Se guarda el nuevo CSV
    df.to_csv(output_csv_path, index = False)

def save_best_model(best_model_path, best_model):
    """
    Función que guarda en disco el estado de un modelo entrenado.

    Args:
        best_model_path (str): Ruta donde se desea guardar el modelo.
        best_model (torch.nn.Module): Modelo ya entrenado.

    Returns:
        None: El modelo se guarda como archivo binario en la ruta especificada.
    """
    # Se guarda el modelo entrenado
    torch.save(best_model.state_dict(), best_model_path)

def save_preds_and_best_model(original_csv_paths, output_csv_paths, best_model_path, predictions, best_model):
    """
    Función que guarda las predicciones en múltiples archivos CSV y almacena el mejor modelo entrenado.

    Args:
        original_csv_paths (list of str): Lista de rutas a los CSV originales.
        output_csv_paths (list of str): Lista de rutas donde se guardarán los CSV con las predicciones.
        best_model_path (str): Ruta donde se guardará el mejor modelo entrenado.
        predictions (list of numpy.ndarray): Lista de arrays de predicciones, uno por CSV.
        best_model (torch.nn.Module): Modelo entrenado a guardar.
    """
    # Se guardan las predicciones del mejor modelo
    for original_path, output_path, prediction in zip(original_csv_paths, output_csv_paths, predictions):
        add_predictions_to_csv(original_path, output_path, prediction)

    # Se guarda el modelo entrenado
    save_best_model(best_model_path, best_model)

### Entrenamiento en audio

In [ ]:
class EmotionDataset(Dataset):
    """
    Dataset personalizado para tareas de reconocimiento de emociones a partir de espectrogramas de Mel, con soporte para etiquetas únicas o múltiples.

    Args:
        df (pandas.DataFrame): DataFrame que contiene las rutas a los espectrogramas y las etiquetas asociadas.
        extra_features_cols (list of str, optional): Lista de columnas con características adicionales numéricas. Si no se especifica, se omiten. Por defecto es None.
        label_cols (str or list of str, optional): Nombre de la columna (para clasificación simple con MELD) o lista de columnas (para clasificación multietiqueta con MOSEI). Por defecto es 'Emotion'.
        multi_label (bool, optional): Indica si la tarea es multietiqueta (True para MOSEI) o clasificación simple (False para MELD). Por defecto es False.

    Methods:
        __len__(): Devuelve el número de muestras en el dataset.
        __getitem__(idx): Devuelve una tupla (mel, extra_features, label) para el índice `idx`.

    Returns:
        tuple:
            torch.Tensor: Tensor del espectrograma de Mel con forma (1, H, W).
            torch.Tensor: Vector de características adicionales o tensor de ceros si no hay.
            torch.Tensor: Etiqueta de clase (entero para MELD) o vector multietiqueta (float para MOSEI).

    Notes:
        - El espectrograma se carga desde la ruta contenida en la columna 'mel_path'.
        - Si `multi_label` es True, se asume que las columnas de etiqueta contienen valores en varias de ellas.
        - Si `multi_label` es False, la etiqueta es un entero representando una única clase emocional.
    """
    def __init__(self, df, extra_features_cols = None, label_cols = 'Emotion', multi_label = False):
        self.df = df
        self.extra_features_cols = extra_features_cols if extra_features_cols is not None else []
        self.label_cols = label_cols
        self.multi_label = multi_label

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Se encuentra la fila a estudiar
        row = self.df.iloc[idx]
        
        # Se carga el Espectrograma de Mel
        mel_path = row['mel_path']
        mel = np.load(mel_path)
        mel = torch.tensor(mel, dtype = torch.float).unsqueeze(0)

        # Se cargan las features
        if self.extra_features_cols:
            extra_features = torch.tensor(row[self.extra_features_cols].astype(np.float32).values, dtype = torch.float)
        else:
            extra_features = torch.zeros(1)

        # Se carga la/s etiqueta/s
        if self.multi_label:
            # Si es MOSEI (7 columnas que van a un vector de emociones)
            label = torch.tensor(row[self.label_cols].astype(np.float32).values, dtype = torch.float)
        else:
            # Si es MELD (1 única columna llamada Emotion)
            label = torch.tensor(np.int64(row[self.label_cols]), dtype = torch.long)

        return mel, extra_features, label

In [ ]:
class EmotionModel(nn.Module):
    """
    Modelo híbrido para reconocimiento de emociones a partir de espectrogramas de Mel, que combina una red CNN con atención y características numéricas adicionales.

    Args:
        backbone_name (str): Nombre del backbone CNN a utilizar. Actualmente soporta 'vgg16' y 'efficientnet_b0'.
        num_extra_features (int, optional): Número de características adicionales extras. Por defecto es 3.
        num_classes (int, optional): Número de clases emocionales de salida. Por defecto es 7.

    Attributes:
        backbone_before_attention (nn.Sequential): Parte inicial del backbone antes de aplicar el primer módulo de atención.
        backbone_after_attention (nn.Sequential): Parte restante del backbone tras aplicar la atención intermedia.
        attention_0 (nn.Sequential): Primer módulo de atención para features intermedias.
        attention_1 (nn.Sequential): Segundo módulo de atención para features más profundas.
        classifier (nn.Sequential): Clasificador final que combina las features del espectrograma y las extra_features.

    Forward Args:
        mel (torch.Tensor): Espectrograma de Mel con forma (batch_size, 1, H, W).
        extra_features (torch.Tensor): Tensor con variables numéricas adicionales por muestra, con forma (batch_size, num_extra_features).

    Returns:
        torch.Tensor: Logits de salida con forma (batch_size, num_classes).

    Notes:
        - El modelo aplica dos módulos de atención: uno a nivel intermedio y otro profundo, ambos aprendidos con convoluciones 1x1.
        - Las features finales se concatenan con `extra_features` antes de pasar por el clasificador.
        - Si se proporciona un backbone no soportado, se lanza un ValueError.
    """
    def __init__(self, backbone_name: str, num_extra_features: int = 3, num_classes: int = 7):
        super().__init__()
        # Se define el nombre del modelo escogido
        self.backbone_name = backbone_name

        if self.backbone_name == "vgg16":
            # Se carga el modelo pre-entrenado
            vgg = models.vgg16(weights = VGG16_Weights.IMAGENET1K_V1)
            
            # Se cambia la primera capa de convolución de 3 canales a un único canal
            vgg.features[0] = nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1)

            # Se divide el entrenamiento entre
            # A) la parte antes de la cuarta convolución
            self.backbone_before_attention = nn.Sequential(*vgg.features[:17])
            # B) la parte a partir de la cuarta convolución
            self.backbone_after_attention = nn.Sequential(*vgg.features[17:])

            # Se definen los canales resultantes local y global
            self.out_channels_local = 256
            self.out_channels_global = 512


        elif self.backbone_name == "efficientnet_b0":
            # Se carga el modelo pre-entrenado
            effnet = models.efficientnet_b0(weights = 'IMAGENET1K_V1')

            # Del primer Sequential, se cambia la primera capa, que es de convolución, de 3 canales a un único canal
            effnet.features[0][0] = nn.Conv2d(1, 32, kernel_size = 3, stride = 2, padding = 1, bias = False)

            # Se divide el entrenamiento entre
            # A) la parte antes de la cuarta convolución
            self.backbone_before_attention = nn.Sequential(*effnet.features[:3])
            # B) la parte a partir de la cuarta convolución
            self.backbone_after_attention = nn.Sequential(*effnet.features[3:])

            # Se definen los canales resultantes local y global
            self.out_channels_local = 24
            self.out_channels_global = 1280

        else:
            raise ValueError(f"Backbone {backbone_name} no soportado aún.")

        # Se definen los módulos de atención
        self.attention_0 = nn.Sequential(nn.Conv2d(self.out_channels_local, self.out_channels_local, kernel_size = 1), nn.Sigmoid())
        self.attention_1 = nn.Sequential(nn.Conv2d(self.out_channels_global, self.out_channels_global, kernel_size = 1),nn.Sigmoid())

        # Se definen las features totales y el clasificador
        total_features = self.out_channels_local + self.out_channels_global + num_extra_features
        self.classifier = nn.Sequential(
            nn.Linear(total_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, mel, extra_features):
        # Se ejecuta hasta la tercera convolución
        x = self.backbone_before_attention(mel)

        # Se ejecuta el módulo de atención 0
        attn0 = self.attention_0(x)

        # Se extraen las features locales del módulo de atención 0
        local_feat0 = torch.mean(x * attn0, dim = [2,3])

        # Se ejecuta la cuarta convolución
        x = self.backbone_after_attention(x)

        # Se ejecuta el módulo de atención 1
        attn1 = self.attention_1(x)
        
        # Se extraen las features locales del módulo de atención 1
        local_feat1 = torch.mean(x * attn1, dim = [2,3])

        # Se concatenan:
        #    - Las features locales del módulo de atención 0
        #    - Las features locales del módulo de atención 1
        #    - Las features externas del dataset
        final_feat = torch.cat([local_feat0, local_feat1, extra_features], dim = 1)

        # Se clasifica la emoción
        logits = self.classifier(final_feat)
        return logits

In [ ]:
def epoch_train_audio(model, dataloader, optimizer, scheduler, loss_fn, device, multilabel = False, verbose = True):
    """
    Función que entrena un modelo con espectrogramas de Mel durante una epoch, manejando tareas de clasificación simple (MELD) o multietiqueta (MOSEI).

    Args:
        model (torch.nn.Module): Modelo a entrenar.
        dataloader (DataLoader): DataLoader que proporciona los batches de entrenamiento (mel, extra_features, labels).
        optimizer (torch.optim.Optimizer): Optimizador utilizado para actualizar los pesos del modelo.
        scheduler (torch.optim.lr_scheduler): Scheduler para ajustar la tasa de aprendizaje.
        loss_fn (callable): Función de pérdida a minimizar.
        device (str): Dispositivo (es 'cuda') donde se ejecuta el entrenamiento.
        multilabel (bool, optional): Si es True, se asume una tarea multietiqueta (MOSEI). Si es False, se asume clasificación única (MELD). Por defecto es False.
        verbose (bool, optional): Si es True, se imprime información sobre la pérdida y métricas al finalizar la época. Por defecto es True.

    Returns:
        tuple:
            float: Loss de la epoch.
            float or None: Accuracy (solo si MELD).
            float: F1-score macro, ajustado a cada tipo de tarea.

    Notes:
        - Para tareas multietiqueta, se utiliza `sigmoid` y un umbral de 0.5 para predicción.
        - Para tareas monoetiqueta, se utiliza `softmax` y `argmax` para obtener la clase más probable.
        - Se utiliza F1-score como métrica principal en ambos casos.
        - Los tensores se acumulan a lo largo de la época para calcular métricas agregadas.
        - El scheduler se actualiza al final de la época.
    """
    # Se pone el modelo en modo entreno
    model.train()

    # Se definen las variables base
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_probs = []
    all_labels = []

    # Por cada espectrograma
    for mel, extra_features, labels in dataloader:
        # Se extrae la información necesaria del dataloader para enviarla al device
        mel = mel.to(device)
        extra_features = extra_features.to(device)
        labels = labels.to(device)

        # Se reinician los gradientes para evitar problemas de gradiente
        optimizer.zero_grad()

        # Forward pass
        outputs = model(mel, extra_features)

        # Se calcula la loss en función del dataset a estudiar
        #    - multilabel = True  --> MOSEI
        #    - multilabel = False --> MELD
        if multilabel:
            loss = loss_fn(outputs, labels.float())
        else:
            loss = loss_fn(outputs, labels)
            
        # Se realiza backpropagation y se actualizan los pesos
        loss.backward()
        optimizer.step()

        # Se acumula la loss
        total_loss += loss.item()

        # Se extraen las predicciones
        if multilabel:
            probs = torch.sigmoid(outputs).detach().cpu()   
            preds = torch.sigmoid(outputs).detach().cpu()
            pred_labels = (preds > 0.5).int()
        else:
            probs = torch.softmax(outputs, dim = 1).cpu()
            preds = torch.argmax(outputs, dim = 1).detach().cpu()
            pred_labels = preds

        labels = labels.cpu()
        all_preds.append(pred_labels)
        all_probs.append(probs)
        all_labels.append(labels)

        # En el caso de MELD, se ajustan las etiquetas
        if not multilabel:
            correct += (pred_labels == labels).sum().item()
            total += labels.size(0)

    # Se actualiza el scheduler
    scheduler.step()

    # Se agrupan todas las predicciones, probabilidades y etiquetas
    all_preds = torch.cat(all_preds)
    all_probs = torch.cat(all_probs)
    all_labels = torch.cat(all_labels)

    all_probs_np = all_probs.detach().cpu().numpy()
    all_labels_np = all_labels.detach().cpu().numpy()

    # Se calcula la métrica global
    #    - AUC-ROC si es MOSEI
    #    - F1-score si es MELD
    if multilabel:
        all_labels_np = (all_labels_np > 0.5).astype(int)
        all_probs_np = (all_probs_np > 0.5).astype(int)
        score = f1_score(all_labels_np, all_probs_np , average = 'macro', zero_division = 0)
        acc = None
    else:
        score = f1_score(all_labels.numpy(), all_preds.numpy(), average = "macro")
        acc = correct / total

    # Se calcula la loss global
    avg_loss = total_loss / len(dataloader)

    # Se muestran los resultados
    if verbose:
        print(f"Train Loss: {avg_loss:.4f}, ACC: {acc:.4f} F1-score: {score:.4f}" if acc is not None else f"Train Loss: {avg_loss:.4f}, f1-score: {score:.4f}")
    return avg_loss, acc, score

In [ ]:
def evaluate_audio(model, dataloader, loss_fn, device, multilabel = False, verbose = True):
    """
    Función que evalúa un modelo sobre espectrogramas de Mel sin actualizar los pesos, para tareas monoetiqueta (MELD) o multietiqueta (MOSEI).

    Args:
        model (torch.nn.Module): Modelo a evaluar.
        dataloader (DataLoader): DataLoader que proporciona los batches de validación o prueba (mel, extra_features, labels).
        loss_fn (callable): Función de pérdida a utilizar.
        device (str): Dispositivo (es 'cuda') sobre el cual se ejecutará la evaluación.
        multilabel (bool, optional): Indica si se trata de una tarea multietiqueta (True para MOSEI) o monoetiqueta (False para MELD). Por defecto es False.
        verbose (bool, optional): Si es True, se imprime por consola la pérdida y métricas al final. Por defecto es True.

    Returns:
        tuple:
            float: Loss durante la evaluación.
            float or None: Accuracy, solo aplicable en tareas monoetiqueta de MELD.
            float: F1-score macro para ambas tareas.

    Notes:
        - Para tareas multietiqueta (MOSEI), se utiliza `sigmoid` y umbral de 0.5 para clasificar cada emoción.
        - Para tareas monoetiqueta (MELD), se utiliza `softmax` y `argmax` para predecir la clase.
        - No se calculan gradientes (`torch.no_grad()`) para acelerar la inferencia y reducir uso de memoria.
        - Se calcula el F1-score macro en ambos casos, y Accuracy en tareas monoetiqueta (MELD).
    """
    # Se manda el modelo a modo evaluación
    model.eval()

    # Se definen todas las variables base
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_probs = []
    all_labels = []

    # Se le establece al modelo que no calcule o guarde los gradientes, así ahorrando memoria 
    # y aumentando la velocidad de las predicciones
    with torch.no_grad():
        # Por cada espectrograma
        for mel, extra_features, labels in dataloader:
            # Se extrae la información necesaria del dataloader para enviarla al device
            mel = mel.to(device)
            extra_features = extra_features.to(device)
            labels = labels.to(device)

            # Forward pass sin backpropagation
            outputs = model(mel, extra_features)

            # Se calcula la loss en función del dataset a estudiar
            #    - multilabel = True  --> MOSEI
            #    - multilabel = False --> MELD
            if multilabel:
                loss = loss_fn(outputs, labels.float())
            else:
                loss = loss_fn(outputs, labels)

            # Se acumula la loss
            total_loss += loss.item()

            # Se extraen las predicciones
            if multilabel:
                probs = torch.sigmoid(outputs).detach().cpu()
                preds = torch.sigmoid(outputs).cpu()
                pred_labels = (preds > 0.5).int()
            else:
                probs = torch.softmax(outputs, dim = 1).cpu()
                preds = torch.argmax(outputs, dim = 1).cpu()
                pred_labels = preds

            labels = labels.cpu()
            all_preds.append(pred_labels)
            all_probs.append(probs)
            all_labels.append(labels)

            # En el caso de MELD, se ajustan las etiquetas
            if not multilabel:
                correct += (pred_labels == labels).sum().item()
                total += labels.size(0)

    # Se agrupan todas las predicciones y etiquetas
    all_preds = torch.cat(all_preds)
    all_probs = torch.cat(all_probs)
    all_labels = torch.cat(all_labels)

    all_probs_np = all_probs.detach().cpu().numpy()
    all_labels_np = all_labels.detach().cpu().numpy()

    # Se calcula la métrica global
    #    - AUC-ROC si es MOSEI
    #    - F1-score si es MELD
    if multilabel:
        all_labels_np = (all_labels_np > 0.5).astype(int)
        all_probs_np = (all_probs_np > 0.5).astype(int)
        score = f1_score(all_labels_np, all_probs_np, average = 'macro', zero_division = 0)
        acc = None
    else:
        score = f1_score(all_labels.numpy(), all_preds.numpy(), average = "macro")
        acc = correct / total
        
    # Se calcula la loss global
    avg_loss = total_loss / len(dataloader)

    # Se muestran los resultados
    if verbose:
        print(f"Val Loss: {avg_loss:.4f}, ACC: {acc:.4f} F1-score: {score:.4f}" if acc is not None else f"Val Loss: {avg_loss:.4f}, AUC: {score:.4f}")
    return avg_loss, acc, score

In [ ]:
def train_and_evaluate_audio(model, train_loader, dev_loader, num_epochs = 10, lr = 1e-4, patience = 2, multilabel = False, verbose = True):
    """
    Función que entrena y evalúa un modelo basado en espectrogramas de Mel para tareas de clasificación monoetiqueta (MELD) o multietiqueta (MOSEI),
    aplicando early stopping y guardando el mejor modelo.

    Args:
        model (torch.nn.Module): Modelo a entrenar y evaluar.
        train_loader (DataLoader): DataLoader con los datos de entrenamiento.
        dev_loader (DataLoader): DataLoader con los datos de validación.
        num_epochs (int, optional): Número máximo de epochs. Por defecto es 10.
        lr (float, optional): Tasa de aprendizaje inicial para el optimizador AdamW. Por defecto es 1e-4.
        patience (int, optional): Número de epochs sin mejora antes de detener el entrenamiento anticipadamente. Por defecto es 2.
        multilabel (bool, optional): Si es True, se asume una tarea multietiqueta (MOSEI). Si es False, se asume monoetiqueta (MELD). Por defecto es False.
        verbose (bool, optional): Si es True, se imprimen métricas y curva de entrenamiento. Por defecto es True.

    Returns:
        tuple:
            torch.nn.Module: Modelo entrenado con los mejores pesos encontrados durante el entrenamiento.
            float: Mejor F1-score macro obtenido en validación.

    Notes:
        - Se usa `CrossEntropyLoss` para clasificación simple (MELD) y `BCEWithLogitsLoss` para clasificación multietiqueta (MOSEI).
        - Se aplica un scheduler `StepLR` para reducir el LR cada 5 epochs (factor 0.5).
        - Se almacena un historial completo de métricas y se puede visualizar mediante `plot_training_history` si verbose es True.
        - Se activa detención temprana si el F1-score en validación no mejora durante `patience` epochs consecutivas.
    """
    # Se manda el modelo al device
    device = 'cuda'
    model = model.to(device)

    # Se define el optimizador AdamW
    optimizer = AdamW(model.parameters(), lr = lr, weight_decay = 1e-4)

    # Se define el scheduler para el LR
    scheduler = StepLR(optimizer, step_size = 5, gamma = 0.5)

    # Se define la función de loss en función del dataset
    #    - multilabel = True  --> MOSEI
    #    - multilabel = False --> MELD
    if multilabel:
        loss_fn = nn.BCEWithLogitsLoss()
    else:
        loss_fn = nn.CrossEntropyLoss()
        
    # Se define la historia
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'train_f1': [], 'val_f1': []}

    # Se define el EalyStopping
    early_stopping = EarlyStopping(patience = patience, verbose = True)

    # Se definen las variables base
    best_f1 = 0
    best_model_state = None
    
    # Por cada epoch
    for epoch in range(num_epochs):
        if verbose:
            print(f"\n Epoch {epoch}")

        # Se entrena el modelo
        train_loss, train_acc, train_score = epoch_train_audio(model, train_loader, optimizer, scheduler, loss_fn, device, multilabel, verbose)

        # Se valida el modelo
        val_loss, val_acc, val_score = evaluate_audio(model, dev_loader, loss_fn, device, multilabel, verbose)

         # Si se encuentra un mejor modelo, se guarda
        if val_score > best_f1:
            best_f1 = val_score
            best_model_state = model.state_dict()
            
        # Se añaden lso resultados a la historia del entrenamiento
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['train_f1'].append(train_score)
        history['val_f1'].append(val_score)


        # Se llama al Early stopper para saber si hay que parar el entrenamiento
        early_stopping(val_score, model)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    # Se guarda el estado del mejor modelo encontrado entre las epochs
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    if verbose:
        # Se plotea la historia del entrenamiento
        plot_training_history(history)

    # Se devuleve le modelo
    return model, best_f1


In [ ]:
def run_audio_model_competition(competition_dict, device = 'cuda', epochs = 5, lr = 1e-4, patience = 2, multilabel = False, verbose = False):
    """
    Función que entrena y compara múltiples modelos de audio basados en espectrogramas de Mel para seleccionar el que obtiene el mejor F1-score en validación.

    Args:
        competition_dict (dict): Diccionario con la forma 
            {'model_name': (modelo, dataloader_entrenamiento, dataloader_validación)}.
        device (str, optional): Dispositivo sobre el cual se ejecutará el entrenamiento. Por defecto es 'cuda'.
        epochs (int, optional): Número de epochs de entrenamiento. Por defecto es 5.
        lr (float, optional): Tasa de aprendizaje para el optimizador AdamW. Por defecto es 1e-4.
        patience (int, optional): Número de epochs sin mejora antes de aplicar early stopping. Por defecto es 2.
        multilabel (bool, optional): Si es True, se asume una tarea multietiqueta (MOSEI); si es False, una tarea monoetiqueta (MELD). Por defecto es False.
        verbose (bool, optional): Si es True, se muestra información detallada del entrenamiento. Por defecto es False.

    Returns:
        tuple:
            str: Nombre del modelo con mejor rendimiento.
            torch.nn.Module: Modelo entrenado correspondiente.
            dict: Diccionario con las métricas del mejor modelo, incluyendo:
                - 'f1_score': F1-score obtenido.
                - 'model': Modelo entrenado.

    Notes:
        - La evaluación se realiza usando `train_and_evaluate_audio` y `predict_audio`.
        - Se guarda el mejor modelo con base en el F1-score macro sobre el conjunto de validación.
        - Para tareas multietiqueta (MOSEI) se utiliza `BCEWithLogitsLoss`; para monoetiqueta (MELD), `CrossEntropyLoss`.
    """
    # Se define el disccionario para los esultados de la competición
    results = {}

    # Por cada modelo que compite
    for model_name, (model, train_dataloader, dev_dataloader) in competition_dict.items():
        print(f"Entrenando modelo {model_name}...")

        # Se define la función de loss en función del dataset
        #    - multilabel = True  --> MOSEI
        #    - multilabel = False --> MELD
        if multilabel:
            loss_fn = nn.BCEWithLogitsLoss()
        else:
            loss_fn = nn.CrossEntropyLoss()

        # Se entrena el modelo
        trained_model, best_f1 = train_and_evaluate_audio(model,
                                                 train_dataloader,
                                                 dev_dataloader,
                                                 num_epochs = epochs,
                                                 lr = lr,
                                                 patience = patience,
                                                 multilabel = multilabel,
                                                 verbose = verbose)

        # Se predice en el conjunto de validación
        y_pred_dev, y_probs_dev = predict_audio(trained_model, dev_dataloader, device)
        
        # Se obtiene las etiquetas reales
        true_labels_dev = []
        for _, _, labels in dev_dataloader:
            true_labels_dev.append(labels)
        true_labels_dev = torch.cat(true_labels_dev).cpu().numpy()
        
        # Se guardan los resultados de la evaluación
        results[model_name] = {
            'f1_score': best_f1,
            'model': trained_model
        }

        print(f"{model_name} - F1-Score: {best_f1:.4f}\n")

    # Se selecciona el mejor modelo
    best_model_name = max(results, key = lambda x: results[x]['f1_score'])
    best_model = results[best_model_name]['model']

    print(f"\nMejor modelo: {best_model_name} con F1-score: {results[best_model_name]['f1_score']:.4f}")

    return best_model_name, best_model, results[best_model_name]


In [ ]:
def grid_search_audio_hyperparameters(model_name, train_dataset, dev_dataset, extra_features, multilabel = False, verbose = False):
    """
    Función que realiza un 'grid search' sobre combinaciones de hiperparámetros para entrenar un modelo de audio basado en espectrogramas de Mel.

    Args:
        model_name (str): Nombre del backbone CNN a utilizar (están soportados 'vgg16' o 'efficientnet_b0').
        train_dataset (Dataset): Dataset de entrenamiento 'EmotionDataset'.
        dev_dataset (Dataset): Dataset de validación 'EmotionDataset'.
        extra_features (int): Número de características adicionales a incluir en el modelo.
        multilabel (bool, optional): Si es True, se trata de una tarea multietiqueta (MOSEI); si es False, es clasificación simple (MELD). Por defecto es False.
        verbose (bool, optional): Si es True, se imprimen detalles del entrenamiento y evaluación. Por defecto es False.

    Returns:
        dict: Diccionario con la mejor combinación de hiperparámetros encontrados, incluyendo:
            - 'batch_size': Tamaño del batch.
            - 'epochs': Número de epochs.
            - 'learning_rate': Tasa de aprendizaje.
            - 'best_f1': Mejor F1-score alcanzado en validación.

    Notes:
        - Se prueban múltiples combinaciones de 'batch_size', 'epochs' y 'learning_rate'.
        - Cada combinación se entrena usando 'train_and_evaluate_audio' y se evalúa con 'evaluate_audio'.
        - Se utiliza 'CrossEntropyLoss' (MELD) o 'BCEWithLogitsLoss' (MOSEI) según el tipo de tarea.
        - El criterio para seleccionar la mejor combinación es el mayor F1-score obtenido sobre el conjunto de validación.
    """
    # Se define el grid de hiperparámetros a probar
    batch_sizes = [32, 64]
    learning_rates = [1e-4, 5e-5]
    epochs_list = [8, 10]

    # Se define la lista para los resultados y el device
    results = []
    device = 'cuda'

    # Por cada combianción
    for batch_size, epochs, lr in product(batch_sizes, epochs_list, learning_rates):
        print(f"\n Combinación a probar: batch = {batch_size}, epochs = {epochs}, LR = {lr}")
        
        # Se define la función de loss en función del dataset
        #    - multilabel = True  --> MOSEI
        #    - multilabel = False --> MELD
        if multilabel:
            loss_fn = nn.BCEWithLogitsLoss()
        else:
            loss_fn = nn.CrossEntropyLoss()

        # Se crean  los dataloaders      
        train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 2)
        dev_dataloader = DataLoader(dev_dataset, batch_size = batch_size, shuffle = False, num_workers = 2)

         # Se define el modelo a estrenar
        model = EmotionModel(backbone_name = model_name, 
                             num_extra_features = extra_features)
        
        # Se entrena el modelo con la combinación escogida
        trained_model, _ = train_and_evaluate_audio(model,
                                                 train_dataloader,
                                                 dev_dataloader,
                                                 num_epochs = epochs,
                                                 lr = lr,
                                                 patience = 2,
                                                 multilabel = multilabel,
                                                 verbose = verbose)

        # Se evalua usando el modelo entrenado
        dev_loss, dev_acc, dev_score = evaluate_audio(trained_model, 
                                                      dev_dataloader, 
                                                      loss_fn = loss_fn, 
                                                      device = device, 
                                                      multilabel = multilabel, 
                                                      verbose = verbose)
        # Se guardan los resultados de la evaluación
        results.append({
            "batch_size": batch_size,
            "epochs": epochs,
            "learning_rate": lr,
            "best_f1": dev_score
        })

    # Se ordenan los resultados de mejor a peor segun su F1-score
    results = sorted(results, key = lambda x: x['best_f1'], reverse = True)

    # Se muestra la mejor combinación encontrada
    print("\nMejor combinaión:")
    print(f"F1-score = {results[0]['best_f1']:.4f} | batch = {results[0]['batch_size']}, epochs = {results[0]['epochs']}, LR = {results[0]['learning_rate']}")

    # Se devuelven los mejores hiperparámetros
    return results[0]

## Funciones auxiliares para la Evaluación

### Evaluación en texto

In [ ]:
def predict(model, dataloader, device):
    """
    Función que genera predicciones a partir de un modelo Transformer entrenado usando un DataLoader con entradas tokenizadas y características adicionales.

    Args:
        model (torch.nn.Module): Modelo entrenado que realiza la inferencia.
        dataloader (DataLoader): DataLoader que contiene los batches de evaluación (input_ids, attention_mask, features).
        device (str): Dispositivo (es 'cuda') sobre el cual se ejecutará la predicción.

    Returns:
        tuple:
            numpy.ndarray: Vector de predicciones (índices de clase).
            numpy.ndarray: Matriz de probabilidades para cada clase, con forma (n_samples, n_classes).

    Notes:
        - El modelo se pone en modo evaluación (`model.eval()`) y se desactiva el cálculo de gradientes con `torch.no_grad()`.
        - Se aplica `softmax` sobre los logits del modelo para obtener las probabilidades por clase.
        - Se usa `argmax` para extraer la clase con mayor probabilidad como predicción final.
        - Los resultados se acumulan por batch y se devuelven como arrays de NumPy.
    """
    # Se pone el modelo a modo evaluación
    model.eval()

    # Se manda el modleo al device
    model.to(device)

    # Se definen las listas de resultados
    all_preds = []
    all_probs = []

    # Se le establece al modelo que no calcule o guarde los gradientes, así ahorrando memoria 
    # y aumentando la velocidad de las predicciones
    with torch.no_grad():
        # Por cada batch
        for batch in dataloader:
            # Se extrae la información necesaria del dataloader para enviarla al device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            features = batch['features'].to(device)

            # Forward pass sin backpropagation
            outputs = model(input_ids = input_ids, 
                            attention_mask = attention_mask, 
                            features = features)

            # Se extraen las probabilidades y las predicciones
            probabilities = torch.softmax(outputs, dim = 1)
            predictions = torch.argmax(probabilities, dim = 1)

            # Se almacenan los resultados
            all_preds.extend(predictions.cpu().numpy())
            all_probs.extend(probabilities.cpu().numpy())

    # Se devuelven las predicciones y probabilidades
    return np.array(all_preds), np.array(all_probs)


In [ ]:
def evaluate_predictions(y_true, y_pred, y_probs, labels_names = None):
    """
    Función que evalúa el rendimiento de un modelo Transformer de clasificación mediante métricas, matriz de confusión y curvas ROC.

    Args:
        y_true (array-like): Etiquetas verdaderas.
        y_pred (array-like): Etiquetas predichas por el modelo.
        y_probs (numpy.ndarray): Matriz de probabilidades por clase, de forma (n_samples, n_classes).
        labels_names (list of str, optional): Nombres de las clases, usados para etiquetar los ejes y leyendas. Si no se proporciona, se usan las clases numéricas.

    Returns:
        None: La función imprime métricas por consola y muestra visualizaciones (matriz de confusión y curvas ROC).

    Notes:
        - Se calculan las siguientes métricas: Accuracy, Precisión, Recall, F1-score (promedio ponderado) y Classification Report completo.
        - Las etiquetas verdaderas se binarizan con `label_binarize` para calcular las curvas ROC multicategoría de MOSEI.
    """
    # Se calculan todas las métricas necesarias
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average = 'weighted', zero_division = 0)
    rec = recall_score(y_true, y_pred, average = 'weighted', zero_division = 0)
    f1 = f1_score(y_true, y_pred, average = 'weighted', zero_division = 0)
    report = classification_report(y_true, y_pred, zero_division = 0)

    # Se muestran las métricas calculadas
    print("\nResultados de la evaluación:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print("\nClassification Report:")
    print(report)

    # Se calcula la matriz de confusión
    cm = confusion_matrix(y_true, y_pred)

    # Se muestra la matriz de confusión
    plt.figure(figsize = (6,5))
    sns.heatmap(cm, annot = True, fmt = 'd', cmap = 'Blues', 
                xticklabels = labels_names if labels_names else np.unique(y_true),
                yticklabels = labels_names if labels_names else np.unique(y_true))
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

    # Se extrae el número de clases predichas para calcular su AUC-ROC
    n_classes = len(labels_names)

    # Se binarizan las etiquetas verdaderas
    y_true_bin = label_binarize(y_true, classes = list(range(n_classes)))

    # Por cada clase, se calcula su curva ROC y se plotea
    plt.figure(figsize = (8,6))
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label = f'{labels_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves')
    plt.legend(loc = "lower right")
    plt.grid()
    plt.show()


### Evaluación en audio

In [ ]:
def predict_audio(model, dataloader, device, multilabel = False):
    """
    Función que genera predicciones a partir de un modelo de audio entrenado usando espectrogramas de Mel y características adicionales.

    Args:
        model (torch.nn.Module): Modelo entrenado que realiza la inferencia.
        dataloader (DataLoader): DataLoader que contiene los batches de evaluación (mel, extra_features, labels).
        device (str): Dispositivo (es 'cuda') sobre el cual se ejecutará la predicción.
        multilabel (bool, optional): Si es True, se asume una tarea multietiqueta (MOSEI); si es False, clasificación única (MELD). Por defecto es False.

    Returns:
        tuple:
            numpy.ndarray: Predicciones del modelo:
                - Array binario para multietiqueta (forma: [n_samples, n_classes]).
                - Array de clases predichas para clasificación simple (forma: [n_samples]).
            numpy.ndarray: Probabilidades para cada clase, con forma (n_samples, n_classes).

    Notes:
        - El modelo se pone en modo evaluación (`model.eval()`).
        - Se desactiva el cálculo de gradientes con `torch.no_grad()` para acelerar la inferencia.
        - Se aplica `sigmoid` + umbral 0.5 para predicciones multietiqueta (MOSEI) y  `softmax` + `argmax`, para predicciones monoetiqueta (MELD).
    """
    # Se pone el modelo a modo evaluación
    model.eval()

    # Se manda el modleo al device
    model.to(device)

    # Se definen las listas de resultados
    all_preds = []
    all_probs = []

    # Se le establece al modelo que no calcule o guarde los gradientes, así ahorrando memoria 
    # y aumentando la velocidad de las predicciones
    with torch.no_grad():
        # Por cada Espectrograma de Mel
        for mel, extra_features, _ in dataloader:
            # Se extrae la información necesaria del dataloader para enviarla al device
            mel = mel.to(device)
            extra_features = extra_features.to(device)

            # Forward pass sin backpropagation
            outputs = model(mel, extra_features)

            # Se extraen las probabilidades y las predicciones con
            if multilabel:
                # Sigmoid si es MOSEI
                probabilities = torch.sigmoid(outputs)
                predictions = (probabilities > 0.5).int()
            else:
                # Softmax si es MOSEI
                probabilities = torch.softmax(outputs, dim = 1)
                predictions = torch.argmax(probabilities, dim = 1)

            # Se almacenan los resultados
            all_preds.append(predictions.cpu().numpy())
            all_probs.append(probabilities.cpu().numpy())

    # Se devuelven las predicciones y probabilidades
    return np.concatenate(all_preds), np.concatenate(all_probs)


In [ ]:
def evaluate_audio_predictions(y_true, y_pred, y_probs, labels_names, multilabel = False):
    """
    Función que evalúa las predicciones de un modelo de audio, mostrando métricas, matriz de confusión y curvas ROC para tareas monoetiqueta (MELD) o multietiquet (MOSEI).

    Args:
        y_true (array-like or pandas.DataFrame): Etiquetas reales. En tareas multietiqueta (MOSEI), debe tener forma (n_samples, n_classes).
        y_pred (array-like): Etiquetas predichas por el modelo. En multietiqueta (MOSEI), es una matriz binaria (n_samples, n_classes).
        y_probs (array-like): Probabilidades predichas por el modelo, con forma (n_samples, n_classes).
        labels_names (list of str): Lista con los nombres de las clases, usada para visualizar etiquetas en gráficas y métricas.
        multilabel (bool, optional): Si es True, se evalúa por clase de forma individual (MOSEI); si es False, se evalúa clasificación única (MELD). Por defecto es False.

    Returns:
        None: La función imprime métricas por consola y muestra visualizaciones (matriz de confusión y curvas ROC).

    Notes:
        - En tareas multietiqueta (MOSEI):
            - Se evalúa Accuracy, Precisión, Recall y F1 por clase.
            - Se plotean las curvas ROC para cada clase de forma individual.
        - En tareas monoetiqueta (MELD):
            - Se calcula el Accuracy global, Precisión, Recall, F1 ponderado y el classification report.
            - Se visualiza la matriz de confusión con etiquetas de clase.
            - Se plotean las curvas ROC por clase tras binarizar las etiquetas reales.
    """
    if multilabel:
        # En el caso de MOSEI, al ser multilabel, se evalúan por separado cada clase
        print("\nResultados multilabel (por clase):")
        y_true = y_true.values
        y_pred = y_pred if isinstance(y_pred, np.ndarray) else np.array(y_pred)
        y_probs = y_probs if isinstance(y_probs, np.ndarray) else np.array(y_probs)
        for idx, label in enumerate(labels_names):
            # Se extraen las predicciones concretas de esta clase
            y_true_class = y_true[:, idx]
            y_pred_class = y_pred[:, idx]

            # Se binariza y_true_class
            y_true_class = (y_true_class >= 0.5).astype(int)

            # Se calculan todas las métricas necesarias
            acc = accuracy_score(y_true_class, y_pred_class)
            prec = precision_score(y_true_class, y_pred_class, zero_division = 0)
            rec = recall_score(y_true_class, y_pred_class, zero_division = 0)
            f1 = f1_score(y_true_class, y_pred_class, zero_division = 0)
            print(f"{label}: ACC = {acc:.4f}, Precision = {prec:.4f}, Recall = {rec:.4f}, F1 = {f1:.4f}")
        
        # Se calcula el AUC de cada clase
        plt.figure(figsize = (8,6))
        print("\nAUC para cada clase:")
        for i in range(y_probs.shape[1]):
            try:
                fpr, tpr, _ = roc_curve(y_true[:, i], y_probs[:, i])
                roc_auc = auc(fpr, tpr)
                print(f"{labels_names[i]} AUC = {roc_auc:.2f}")
                plt.plot(fpr, tpr, label = f'{labels_names[i]} (AUC = {roc_auc:.2f})')
            except:
                print(f"{labels_names[i]} AUC no disponible")

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves Multilabel')
        plt.legend(loc = "lower right")
        plt.grid()
        plt.show()

    else:
        # En el caso de MELD, al solo tener un único valor a predeci, se evalúa todo junto
        # Se calculan todas las métricas necesarias
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average = 'weighted', zero_division = 0)
        rec = recall_score(y_true, y_pred, average = 'weighted', zero_division = 0)
        f1 = f1_score(y_true, y_pred, average = 'weighted', zero_division = 0)
        report = classification_report(y_true, y_pred, target_names = labels_names, zero_division = 0)

        print("\nResultados de la evaluación:")
        print(f"Accuracy:  {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall:    {rec:.4f}")
        print(f"F1 Score:  {f1:.4f}")
        print("\nClassification Report:")
        print(report)

        # Se calcula la matriz de confusión
        cm = confusion_matrix(y_true, y_pred)

        # Se muestra la matriz de confusión
        plt.figure(figsize = (8,6))
        sns.heatmap(cm, annot = True, fmt = 'd', cmap = 'Blues', 
                    xticklabels = labels_names if labels_names else np.unique(y_true),
                    yticklabels = labels_names if labels_names else np.unique(y_true))
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        plt.title('Confusion Matrix')
        plt.show()

        # Se extrae el número de clases predichas para calcular su AUC-ROC
        n_classes = len(labels_names)

        # Se binarizan las etiquetas verdaderas
        y_true_bin = label_binarize(y_true, classes = list(range(n_classes)))

        # Por cada clase, se calcula su curva ROC y se plotea
        plt.figure(figsize = (8,6))
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label = f'{labels_names[i]} (AUC = {roc_auc:.2f})')

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves')
        plt.legend(loc = "lower right")
        plt.grid()
        plt.show()
